In [1]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, KeepTogether, PageBreak, Flowable
from reportlab.platypus import Spacer, Image, Table, TableStyle,  ListFlowable, ListItem
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from bs4 import BeautifulSoup

class CustomDocTemplate(SimpleDocTemplate):
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.header_text = ""  # Initialize header text attribute

    def set_header_text(self, text):
        self.header_text = text

def footer(canvas, doc):
    # canvas.saveState()

    # footer_text = (
    #     "Association pour la Sauvegarde du Léman -- www.asleman.org "
    #     "-- Tél : +41 22 736 86 20 -- Email : "
    # )
    # email_text = "asl@asleman.org"
    page_number = f"{doc.page}"

    # # Footer
    footer_line_y_position = 1.6 * cm  # Position the line just above the footer text
    canvas.setStrokeColor(colors.grey)  # Set the color to gray
    canvas.setLineWidth(1)  # Set the line width
    canvas.line(1 * cm, footer_line_y_position, 20 * cm, footer_line_y_position)  # Draw the line from left to right


    # canvas.setFont('Helvetica', 8)
    # # canvas.setFillColorRGB(0, 0, 1)  # Set color to blue (RGB: 0, 0, 1)
    # canvas.drawString(1 * cm, 0.75 * cm, footer_text)

    # # Email link
    # email_x_position = 1 * cm + canvas.stringWidth(footer_text, 'Helvetica', 8)
    # canvas.linkURL(f"mailto:{email_text}", (email_x_position, 1 * cm, email_x_position + canvas.stringWidth(email_text, 'Helvetica', 8), .75 * cm), relative=1)
    # canvas.drawString(email_x_position, .75 * cm, email_text)
    
    # # Page number
    # canvas.drawRightString(20 * cm, .75 * cm, page_number)

    # canvas.restoreState
    canvas.saveState()
    canvas.setFont('Helvetica', 8)
    
    footer_text = "Association pour la Sauvegarde du Léman -- www.asleman.org -- Tél : +41 22 736 86 20 -- Email : asl@asleman.org"
    footer_width = canvas.stringWidth(footer_text, 'Helvetica', 8)
    footer_x = (doc.pagesize[0] - footer_width) / 2
    footer_y = 1 * cm

    canvas.drawString(footer_x, footer_y, footer_text)

    # Add links (website and email)
    website_start = footer_x + canvas.stringWidth("Association pour la Sauvegarde du Léman -- ", 'Helvetica', 8)
    website_width = canvas.stringWidth("www.asleman.org", 'Helvetica', 8)
    canvas.linkURL("http://www.asleman.org", (website_start, footer_y, website_start + website_width, footer_y + 8), relative=0)

    email_start = footer_x + canvas.stringWidth("Association pour la Sauvegarde du Léman -- www.asleman.org -- Tél : +41 22 736 86 20 -- Email : ", 'Helvetica', 8)
    email_width = canvas.stringWidth("asl@asleman.org", 'Helvetica', 8)
    canvas.linkURL("mailto:asl@asleman.org", (email_start, footer_y, email_start + email_width, footer_y + 8), relative=0)
    # Page number
    canvas.drawRightString(20 * cm, .75 * cm, page_number)

    canvas.restoreState()

def header_footer(canvas, doc):
    canvas.saveState()


    # header_text = "Header Text"
    # canvas.setFont('Helvetica-Bold', 12)
    canvas.setFont('Helvetica', 8)
    canvas.drawString(1 * cm, 28.2 * cm,  doc.header_text)  # Positioned inside the top margin
    
    # Add space below the header
    header_height = 28.2 * cm
    space_after_header = .1 * cm  # The amount of space you want below the header
    canvas.drawString(1 * cm, header_height - space_after_header, "")  # This simply moves the starting point down
    

    line_y_position = 28 * cm  # Position the line just below the header text
    canvas.setStrokeColor(colors.grey)  # Set the color to gray
    canvas.setLineWidth(1)  # Set the line width
    canvas.line(1 * cm, line_y_position, 20 * cm, line_y_position)  # Draw the line from left to right

    
    # Footer text
    footer_text = (
        "Association pour la Sauvegarde du Léman -- www.asleman.org "
        "-- Tél : +41 22 736 86 20 -- Email : "
    )
    email_text = "asl@asleman.org"
    page_number = f"{doc.page}"
    
    footer_line_y_position = 1.6 * cm  # Position the line just above the footer text
    canvas.setStrokeColor(colors.grey)  # Set the color to gray
    canvas.setLineWidth(1)  # Set the line width
    canvas.line(1 * cm, footer_line_y_position, 20 * cm, footer_line_y_position)  # Draw the line from left to right



    # Footer
    canvas.setFont('Helvetica', 8)
    # canvas.setFillColorRGB(0, 0, 1)  # Set color to blue (RGB: 0, 0, 1)
    canvas.drawString(1 * cm, 0.75 * cm, footer_text)

    # Email link
    email_x_position = 1 * cm + canvas.stringWidth(footer_text, 'Helvetica', 8)
    canvas.linkURL(f"mailto:{email_text}", (email_x_position, 1 * cm, email_x_position + canvas.stringWidth(email_text, 'Helvetica', 8), .75 * cm), relative=1)
    canvas.drawString(email_x_position, .75 * cm, email_text)
    
    # Page number
    canvas.drawRightString(20 * cm, .75 * cm, page_number)

    canvas.restoreState()

def image_table(rl_image, acaption, image_width):
    caption = Paragraph(acaption, caption_style)
    table_data = [
        [image],     
        [caption]     
    ]
    table = Table(table_data, colWidths=[image_width])
    table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (0, 0), 'CENTER'),  # Center the image
        ('ALIGN', (0, 1), (0, 1), 'LEFT'),    # Left-align the caption under the image
        ('VALIGN', (0, 0), (0, 1), 'MIDDLE'), # Vertically center the image and caption
        ('TOPPADDING', (0, 0), (0, 0), 0),
        ('BOTTOMPADDING', (0, 0), (0, 0), 5),
        ('BOTTOMPADDING', (0, 1), (0, 1), 0),
    ]))

    return table

def add_paragraph_and_spacer(p_text, content):
    par = Paragraph(p_text, body_text_style)
    content.append(par)
    content.append(Spacer(1, 6))
    return content

def add_title(t_text, content):
    title = Paragraph(t_text, title_style)
    content.append(title)
    return content

def add_section_title(s_text, content):
    section_header = Paragraph(s_text, section_title_style)
    content.append(section_header)
    return content

def add_sub_section_title(s_text, content):
    section_header = Paragraph(s_text, subsection_title_style)
    content.append(section_header)
    return content

def new_document( pdf_file, pagesize=A4, rightMargin=2 * cm, leftMargin=2 * cm, topMargin=1.75 * cm,  bottomMargin=1.75 * cm):
    document = CustomDocTemplate(
        pdf_file,
        pagesize=pagesize,
        rightMargin=rightMargin,
        leftMargin=leftMargin,
        topMargin=topMargin,
        bottomMargin=bottomMargin
    )

    return document

def make_a_list(list_items, bulletType='bullet', start='*', bulletFontName='Helvetica', bulletFontSize=12, bulletOffsetY=0, leftIndent=.5*cm, bulletIndent=1*cm):

    bulleted_list = ListFlowable(
    [ListItem(Paragraph(item, body_text_style), leftIndent=bulletIndent) for item in list_items],
    bulletType=bulletType,  
    start=start,            
    bulletFontName=bulletFontName, 
    bulletFontSize=bulletFontSize,
    bulletOffsetY=bulletOffsetY,
    leftIndent=leftIndent)

    return bulleted_list

def get_color_for_value(value, min_value, max_value):
    # Normalize the value to a range between 0 and 1
    normalized_value = (value - min_value) / (max_value - min_value)

    r = 1  # Red is constant
    g = 1 - normalized_value  
    b = 1 - normalized_value  

    return colors.Color(r, g, b)

def create_gradient_table(data):
    # Extract the numeric values from the table (excluding the headers)
    # Convert strings to floats, replacing commas with periods, and update the table
    numeric_values = []
    for row in data[1:]:  # Skip the header row
        for i in range(1, len(row)):  # Skip the row label
            cell = row[i]
            if cell.replace(',', '.').replace('.', '', 1).isdigit():
                value = float(cell.replace(',', '.'))
                row[i] = value  # Update the table with the float value
                numeric_values.append(value)
    
    min_value = min(numeric_values)
    max_value = max(numeric_values)


    table_data = []
    for row in data:
        new_row = []
        for cell in row:
            if isinstance(cell, float):  # Now check if it's already a float
                color = get_color_for_value(cell, min_value, max_value)
                new_row.append(cell)
            else:
                color = colors.white  # Default color for non-numeric cells
                new_row.append(cell)
        table_data.append(new_row)
    
    
    table = Table(table_data)
    style = TableStyle([
        ('GRID', (0, 0), (-1, -1), .5, colors.black),
    ])
  
    
    for row_idx, row in enumerate(table_data[1:], start=1):
        for col_idx, cell in enumerate(row[1:], start=1):
            if isinstance(cell, float):
                color = get_color_for_value(cell, min_value, max_value)
                style.add('BACKGROUND', (col_idx, row_idx), (col_idx, row_idx), color)


    table.setStyle(style)
    return table





class RotatedTableWithCaption(Flowable):
    def __init__(self, table, x, y, angle=-90):
        Flowable.__init__(self)
        self.table = table
        self.x = x
        self.y =y
        self.angle = angle

    def draw(self):
        self.canv.saveState()

        self.canv.rotate(self.angle)

        table_width, table_height = self.table.wrap(0, 0)
     
        self.table.drawOn(self.canv, self.x, self.y)

        self.canv.restoreState()
    
styles = getSampleStyleSheet()
title_style = ParagraphStyle(name='Title', parent=styles['Title'], spaceAfter=12, spaceBefore=0)
caption_style = ParagraphStyle(name='Caption', parent=styles['BodyText'], alignment=0, fontName='Helvetica-Oblique', fontSize=8, leading=10)
default_table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # Header background color
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),  # All text color to black
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Header font
    ('FONTSIZE', (0, 0), (-1, 0), 10),  # Set font size for the header row
    ('FONTSIZE', (0, 1), (-1, -1), 8),  # Set font size for the rest of the table
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),  # Add padding to the header
    ('GRID', (0, 0), (-1, -1), .5, colors.black),  # Grid lines
    ('BACKGROUND', (0, 1), (-1, 1), colors.whitesmoke),  # Alternating row colors
    ('BACKGROUND', (0, 2), (-1, 2), colors.white),  # Alternating row colors
])

default_table_style_small_header = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # Header background color
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),  # All text color to black
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Header font
    ('FONTSIZE', (0, 0), (-1, 0), 8),  # Set font size for the header row
    ('FONTSIZE', (0, 1), (-1, -1), 8),  # Set font size for the rest of the table
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),  # Add padding to the header
    ('GRID', (0, 0), (-1, -1), .5, colors.black),  # Grid lines
    ('BACKGROUND', (0, 1), (-1, 1), colors.whitesmoke),  # Alternating row colors
    ('BACKGROUND', (0, 2), (-1, 2), colors.white),  # Alternating row colors
])

body_text_style = ParagraphStyle(
    name='BodyTextWithBlueLinks',
    parent=styles['BodyText'],
    textColor=colors.black,  # Default text color
    allowWidows=1,
    allowOrphans=1,
    linkUnderline=1,  # Underline links
    fontName='Helvetica',
    fontSize=10,
    leading=14
)

section_title_style = ParagraphStyle(
    name='SectionTitle',
    parent=styles['Heading1'],  # Base this on the Heading1 style
    fontSize=14,
    leading=14,
    spaceAfter=12,
    spaceBefore=4,
    textColor=colors.HexColor('#003366')
)
subsection_title_style = ParagraphStyle(
    name='SubsectionTitle',
    parent=styles['Heading2'],  # Base this on the Heading2 style
    fontSize=12,
    leading=12,
    spaceAfter=8,
    spaceBefore=4,
    textColor=colors.HexColor('#6699CC')
)

In [2]:
# title page
contenti = []





image_path = "resources/images/coverimage.png" 
image_width = 17.6* cm
image_height = 24.6* cm
image = Image(image_path, width=image_width, height=image_height)  
    
# a_caption = (
#     "<b>Figure 2 : </b>Organisation des différentes phases du projet Pla'stock. Les prélèvements de macroplastiques "
#     "ont eu lieu entre février et novembre 2022. Les échantillons de substrats pour le recensement des microplastiques "
#     "ont été prélevés en novembre 2021, puis traités à l’UNIGE entre février 2022 et juin 2023 pour être comptabilisés à "
#     "l’ASL entre mars 2022 et août 2023."   
    
# )

# table_image = image_table(image, a_caption, image_width)
contenti.append(image)
contenti.append(PageBreak())
contenti = add_title("Pla'stock", contenti)
contenti = add_section_title("Étude du stock de plastiques sur les plages du Léman", contenti)
pi = (
    "Réalisée en collaboration avec l’Université de Genève, l’étude «Pla’stock » de <a href='https://asleman.org/' color='blue'>l'Association pour la Sauvegarde du Léman</a>, "
    "proposée et soutenue par la CIPEL afin de répondre à son plan d’action, a permis d’estimer les quantités de plastiques "
    "présentes sur les plages du Léman en 2021 et 2022."
)
contenti = add_paragraph_and_spacer(pi, contenti)

contenti = add_paragraph_and_spacer("<b>Image :</b> Bénévoles en action", contenti)

contenti = add_paragraph_and_spacer("<b>Auteurs :</b> Alexis Pochelon (ASL), Raphaëlle Juge (ASL), Roger Erismann, Serge Stoll (UNIGE), Mathieu Coster (CIPEL)", contenti)

pii = (
    "<b>Avec le soutien de :</b> Fonds Electricité Vitale Vert, Université de Genève, CIPEL, Loterie Romande, Fondation Valery"
)

contenti = add_paragraph_and_spacer(pii, contenti)
contenti = add_paragraph_and_spacer("<b>Accompagnement :</b> Groupe de travail de l'ASL et conseil scientifique de la CIPEL", contenti)

contenti = add_paragraph_and_spacer("<b>Coordination :</b> Suzanne Mader-Feigenwinter (ASL), Serge Stoll (UNIGE) et Nicole Gallina (CIPEL)", contenti)

contenti = add_sub_section_title("Remerciements :", contenti)

pi2 = (
    "<b>Membres du groupe 'plastiques' de l’ASL :</b> Adrien Bonny, Julien Boucher, Jean-Marcel Dorioz, Claude Ganty, Suzanne Mader-Feigenwinter, "
    "Pascal Mulattieri, et Serge Stoll, ainsi qu’Alix Chatton et Malika Sekkiou (stagiaires)"
)

pi3 = (
    "<b>Membres du conseil scientifique et du secrétariat de la CIPEL</b>"
)

pi4 = (
    "<b>Etudiantes et étudiant de l’Université de Genève :</b> Jade Bouzy, Mathis Ducleon, Lea Früh, Coline Guignier et Louise Uhlmann"
)

pi5 = (
    "<b>Bénévoles :</b>  Denise Adler, Anne Affolter, Nadine Allal, Herve Amigo, Elodie Balistreri, Philippe Barbier, Jean-Claude Barrère, "
    "Claudine Battistolo, Dominique Baudet, Alexandre Berthoud, Thierry Bigler, Coralie Bonvin, Michel Borter, Sandrine Bouillot, Nicolas Bourquin, "
    "Karen Brunel, François Bujard, Mathilde Bussy, Gabrielle Chaizy, Sarah Chapalay, Sandra Christen, Flavia Cioffoletti, Julien Claude, "
    "Lionel Combebias, Christelle Crépin, Anastasia de Franchi, Yann Demont, Alain Dériaz, Nathalie Dewarrat, Cédric Drevet, Anne-Marie Dufour, "
    "Dominique Favre, Gilles Favre, Katia Fiaux, Willy Fink, Patrizia Fioriti, Patricia Fontaine, Claire Fouassier, Liliane Fourel, Christophe Gabriel, "
    "Monique Gasnier, Basile Gillard, Pawel Grabarz, Sandra Graf, Nicolas Guillemaux, Sylvain Guironnet, Catherine Herard, Martin Hesener, "
    "Dominique Imhof, Laure Jacquemard, May Kassem, Yvonne Kessler, Brigitte Klinkenbijl, Pierre Kremer, Aurélie Kropf, Catherine Lacour, "
    "Corinne Lavanchy, Françoise Le Fort, Gabrielle Lods, Martine Logossou-Jacot, Yves Lottaz, Michelle  Loup, Anne-Laure Maire, Adrien Maire, "
    "Sébastien Marquis, Philippe Meister, Valérie Mérat, Véronique Merkly, Marianne Michaud, Caroline Michaud, Hassan Mohammad, Martine Moret, "
    "Isabel Muhlethaler, Viviane Murat, Thomas Nook, Martine Oppliguer, Marianne Perron-Martel, Jean-François Pingoud, Danièle Pittet, Johanna "
    "Porchet, Coralie Probst, Aleksandra Racz, Marion Raffini, Philippe Regnault, Séverine Roth, Pierluigi Ruggeri, Michel Sallin, Murielle Salmon, "
    "Cristian Santana, Flore Seigneur, Thatcher Shellaby, Jacqueline Simonin, Ekaterina Souhodolskaya, Frédéric Soulignac, Christine Sullenger, "
    "Sandrine Teutschmann, Madeleine Thévoz, Thierry Maréchal, Jennifer Tschopp, Hans von Zinkernagel, Béatrice Walker, Jake Wellian, Olivier Wenger et Claire Ziegeler. "
)

contenti = add_paragraph_and_spacer(pi2, contenti)
contenti = add_paragraph_and_spacer(pi3, contenti)
contenti = add_paragraph_and_spacer(pi4, contenti)
contenti = add_paragraph_and_spacer(pi5, contenti)
intro = new_document('intro.pdf')
intro.build(contenti)

## Resumé

In [3]:
pdf_file = "chap_1.pdf"

# chap_1 = new_document(pdf_file)
content = []

content = add_title("Pla'stock", content)
content = add_section_title("Resumé", content)

pone = (
    "Issue d'une collaboration entre la CIPEL, l'Université de Genève et l'ASL, "
    "l’étude de science participative « Pla’stock » a permis d’estimer les quantités "
    "de plastiques présentes sur les plages du Léman en 2021 et 2022. Répartis sur 25 plages, "
    "217 échantillons de substrat ont été prélevés. Les microplastiques ont été extraits par "
    "flottaison dans une solution de Iodure de Sodium et comptabilisés à l’aide d’une loupe binoculaires, "
    "permettant de déterminer une moyenne de 7’600 ± 8’300 particules de microplastiques (0,3 mm à 5 mm) "
    "par m², dont 60% de fibres textiles. Par ailleurs, plus de 100 bénévoles formés se sont mobilisés sur "
    "les 25 plages pour recenser les plastiques visibles à l’œil nu (ci-après macroplastique) selon un "
    "protocole strict lors de quatre sessions d’échantillonnage à intervalle de trois mois. "
    "La concentration moyenne en macroplastiques est de 3,42 ± 3,66 éléments plastique par mètre linéaire. "
    "La majorité des éléments récoltés étaient fortement fragmentés et de petite taille (<2,5 cm). "
    "Les emballages de nourriture, les mégots de cigarettes et les pellets (granulés plastiques industriels) "
    "sont les trois éléments les plus souvent identifiés. Une grande variabilité est observable entre les "
    "différentes plages. Deux plages se démarquent des autres par leur abondance en déchets plastiques, "
    "tant pour les macroplastiques que les microplastiques : les plages des Grangettes et du Bouveret. "
    "Leur proximité avec l’embouchure du Rhône pourrait en être la cause."
)
content = add_paragraph_and_spacer(pone, content)
content.append(Spacer(1, 12))
# ptwo = (
#     "The “Pla’stock” participatory science study, a collaboration between CIPEL, "
#     "University of Geneva and ASL, estimated the quantity of plastic on the beaches of Lake Geneva in 2021. "
#     "217 substrate samples from 25 beaches were analysed, giving an average of 7’600 ± 8’300 microplastic "
#     "particles (0.3 mm to 5 mm) per square meter. 60% of the microplastic particles were textile fibers. "
#     "In addition, over one hundred trained volunteers counted the macroplastic debris (fragments visible "
#     "to the naked eye) across the same 25 beaches . This study followed a strict protocol over four sampling "
#     "sessions at three-month intervals. The concentration of macroplastics was 3,42 ± 3,66 plastic elements "
#     "per linear meter. The majority of the elements collected were small and highly fragmented – 0.5 to 2.5 cm. "
#     "The three most common items were: food packaging, cigarette butts and pellets (granulated industrial plastic). "
#     "There was a considerable difference between the different beaches. Two beaches with particularly high levels "
#     "of both macroplastics and microplastics stood out from the rest  - the beaches of Grangettes and Bouveret. "
#     "Their proximity to the mouth of the Rhône could be the cause."    
# )

# content = add_paragraph_and_spacer(ptwo,content)

image_path = "resources/images/pdfimages/CarteMacroMicro.png"  
image_width = 13 * cm 
image = Image(image_path, width=image_width, height=image_width * 3/4) 

a_caption = (
    "<b>Figure 1 :</b> Synthèse des concentrations en plastiques sur les plages du Léman. Les plages du Bouveret et des Grangettes se démarquent fortement. "
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
chap1content = content.copy()
content.append(PageBreak())

# chap_1.build(content, onFirstPage=footer, onLaterPages=header_footer)

## Introduction

In [4]:
# introduction

pdf_file = "chap_2.pdf"
# chap_2 = new_document(pdf_file)
# content = []

content = add_title("Introduction", content)

pone = (
    "La problématique du plastique au niveau mondial représente un des enjeux environnementaux les plus importants "
    "du XXIème siècle. En 2011, une première étude de l’École polytechnique fédérale de Lausanne "
    "(<a href='https://www.epfl.ch/about/' color='blue'> EPFL </a>) a prouvé la présence de microplastiques dans les eaux lémaniques "
    "(Faure et al., 2014). L’apparition de cette catégorie de polluants a fait l’objet de plusieurs études"
    "(Osman et al., 2023) qui ont démontré que les microplastiques sont présents dans tous les compartiments de "
    "l’environnement. Le Léman, plus grand lac d’Europe par son volume, n’est pas épargné. Selon une étude mandatée "
    "par l’Association pour la Sauvegarde du Léman (ASL) en 2018 (Boucher et al., 2019), environ 50 tonnes "
    "de plastiques aboutissent dans le Léman chaque année. La grande majorité de ces derniers (60%) sont des microplastiques "
    "issus de l’abrasion des pneus sur les routes (Boucher et al, 2019). Pour autant, les particules de "
    "pneus sont bien souvent émises à des tailles trop petites pour être détectées par la méthodologie appliquée dans cette "
    "étude. Les emballages et le littering constituent une autre source importante de déchets plastiques (pas loin de  20%). "
    "La Commission Internationale pour la protection des eaux du Léman (<a href='https://www.cipel.org/en/' color='blue'> CIPEL </a>) "
    "a émis trois pistes de réflexion pour étudier la pollution du Léman par les microplastiques figurant dans son plan "
    "d’action 2021-2030 :"
)

content = add_paragraph_and_spacer(pone, content)

list_items = [
    "Évaluer les apports de microplastiques au lac par les affluents et/ou via des déversements par temps de pluie.",
    "Surveiller les impacts sur l’écosystème lacustre (exposition de l’écosystème) au moyen d’analyses des tubes digestifs de poissons",
    "Évaluer le stock de plastiques sur les rives et les plages par une démarche participative."
]

bulleted_list = make_a_list(list_items)
content.append(bulleted_list)
content.append(Spacer(1, 12))

ptwo = (
    "Pour satisfaire à cette troisième orientation, la CIPEL a proposé à l’ASL de développer un projet d’étude pour "
    "quantifier les stocks de plastiques sur les plages du Léman. L’étude proposée par l’ASL traite tant les micro- "
    "(<5 mm) que les macroplastiques. Dans le cadre de ce projet de recherche dit participatif, la société civile a "
    "été mise à contribution et s’est vu confier la partie de l’étude portant sur la récolte et la caractérisation "
    "des macroplastiques selon un protocole prédéterminé. Le but est de contribuer à la sensibilisation du grand "
    "public à la thématique de la pollution des eaux par les plastiques."
)

content = add_paragraph_and_spacer(ptwo, content)


pthree = (
    "L’étude des microplastiques a, quant à elle, été menée conjointement par l’équipe scientifique de l’ASL et deux "
    "étudiantes réalisant leur travail de master à l’Université de Genève ([UNIGE](https://www.unige.ch/)) au sein du "
    "groupe du Dr. Serge Stoll, spécialiste en physique et chimie de l’environnement "
    "(<a href='https://www.unige.ch/forel/' color='blue'>Département F.-A. Forel</a>)."
)

content = add_paragraph_and_spacer(pthree, content)
content.append(PageBreak())
chap2content = content.copy()
# chap_2.build(content, onFirstPage=footer, onLaterPages=header_footer)

## Déroulement du projet

In [5]:
pdf_file = "chap_3.pdf"
chap_3 = new_document(pdf_file)

# content = []

content = add_title("Déroulement du projet", content)
content.append(Spacer(1, 12))  # Add space between the title and the first paragraph

pfour = (
    'La réalisation de l’étude des stocks de plastiques sur les plages lémaniques s’est déroulée en cinq phases (Figure 2).'
    
)

content = add_paragraph_and_spacer(pfour, content)

image_path = "resources/images/diagramme.jpg" 
image_width = 18* cm
image_height = 18.2* cm
image = Image(image_path, width=image_width, height=image_height)  
    
a_caption = (
    "<b>Figure 1 : </b>Organisation des différentes phases du projet Pla'stock. Les prélèvements de macroplastiques "
    "ont eu lieu entre février et novembre 2022. Les échantillons de substrats pour le recensement des microplastiques "
    "ont été prélevés en novembre 2021, puis traités à l’UNIGE entre février 2022 et juin 2023 pour être comptabilisés à "
    "l’ASL entre mars 2022 et août 2023."   
    
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
chap3content = content.copy()
content.append(PageBreak())
# chap_3.build(content, onFirstPage=footer, onLaterPages=header_footer)

## Questions de recherche

In [6]:
pdf_file = "chap_4.pdf"
chap_4 = new_document(pdf_file)
# content = []
 
content = add_title("Questions de recherche", content)
pfive = (
    'L’élaboration du plan de recherche pour la caractérisation des stocks de plastique qui '
    'se sont déposés sur les plages du Léman est basée sur la formulation d’un certain nombre de questions.'
    
)

content = add_paragraph_and_spacer(pfive, content)

psix = (
    "La nature (type de polymère), la forme, l’origine, la taille et l’abondance des microplastiques et des "
    "macroplastiques qui se déversent sur les plages du Léman présentent-elles des différences significatives "
    "selon les plages considérées ? Si oui, quels facteurs sont-ils susceptibles d’expliquer ces différences ?"
)

content = add_paragraph_and_spacer(psix, content)

pseven = (
    "Parmi ceux qui semblent être les plus déterminants en ce qui concerne les sites sur lesquels les prélèvements "
    "sont effectués, citons :"
)

content = add_paragraph_and_spacer(pseven, content)
    
list_items = [
    "la localisation, à savoir le lieu géographique et son environnement immédiat le profil de la rive : linéaire, baie plus ou moins ouverte",
    "l’exposition au soleil, aux vents, aux vagues",
    "la fréquentation par le public",
    "le degré d’urbanisation alentours",
    "le substrat sur la ligne d’eau et sur la plage"
]

bulleted_list = make_a_list(list_items)
content.append(bulleted_list)
content.append(Spacer(1, 12))

peight = (
    "Comment se situent les résultats de cette étude par rapport à ceux d’autres travaux réalisés "
    "autour du Léman ou ailleurs en Suisse, en Europe ou dans le monde ?"
)

content = add_paragraph_and_spacer(peight, content)

pnine = (
    "Si les déchets ne sont pas trop fragmentés, il est possible d’en déterminer la source. "
    "Il sera intéressant de définir quels sont les objets les plus fréquemment trouvés sur les plages ? "
)

content = add_paragraph_and_spacer(pnine, content)

pten =(
    "Contrairement aux microplastiques dont l’origine ne peut être déterminée, celle des macroplastiques peut "
    "l’être dans la mesure où ceux-ci ne sont pas trop fragmentés. Dans ce cas, il sera intéressant de "
    "définir quels sont les objets les plus fréquemment trouvés sur les plages."
    
)

content = add_paragraph_and_spacer(pten, content)
chap4content = content.copy()
# chap_4.build(content, onFirstPage=footer, onLaterPages=header_footer)
content.append(PageBreak())

## Méthodes

In [7]:
pdf_file = "chap_5.pdf"
chap_5 = new_document(pdf_file)

chap_5.set_header_text("Pla'stock: Méthodes")

# content = []

content = add_title("Méthodes", content)

p11 = (
    "Que ce soit pour les macro- ou les microplastiques, les méthodes de prélèvements et d’analyses ont été développées "
    "puis testées et validées par l’ASL. Elles sont majoritairement compatibles avec les protocoles scientifiques utilisés "
    "par les différentes études ayant traité de la même thématique en Europe (Gerdts, 2019) et au niveau helvétique "
    "(OFEV, 2021),(Swiss Litter Report, 2018). La méthodologie appliquée ici aux macroplastiques se démarque des autres "
    "études par la prise en compte du mètre carré comme unité de référence par rapport à d’autres études qui se basent "
    "sur le mètre linéaire de ligne d’eau. Ce choix a été effectué afin de pouvoir comparer les abondances par unité de "
    "surface entre les microplastiques et les macroplastiques. La méthodologie de prélèvements et d’analyses pour "
    "les microplastiques a été testée et choisie pour être compatible avec les protocoles d’autres études européennes aux "
    "fins de comparaison (Faure et al., 2018), (Frei et al, 2022). Les méthodes ont été validées par le groupe de "
    "travail « Pla’stock » de l’ASL et le Conseil scientifique de la CIPEL."
)

content = add_paragraph_and_spacer(p11, content)

content = add_section_title("Zone d'étude", content)


p12 = (
    "Sur la base du périmètre défini par la CIPEL, les 200 km de rives du Léman ont été découpés en 25 tronçons de longueur "
    "équivalente. Une plage a été choisie dans chacun de ces secteurs selon ses caractéristiques (type de substrat, "
    "fréquentation, degré d’urbanisation des alentours) et la diversité des milieux qui la constituent, et ce, afin d’obtenir "
    "un échantillonnage représentatif de la réalité. Une partie des plages correspond à celles inventoriées dans le cadre de "
    "l’étude (OFEV, 2021)."
    
)

content = add_paragraph_and_spacer(p12, content)

image_path = "resources/images/pdfimages/secteur_detude.jpg"  
image_width = 16* cm
image_height = 11.2 * cm
image = Image(image_path, width=image_width, height=image_height)  
    
# Add caption centered under the image
a_caption = (
    "<b>Figure 3 :</b> Séparation des 200 km de rives lémaniques en 25 secteurs pour le choix des plages de l’étude."   
    
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
content.append(Spacer(2, 12))

p13 = (
    "Le choix s’est porté sur neuf grandes plages de plus de 80 m de long, neuf de 50 à 80 m et sept de 15 à 50 m. "
    "Les surfaces inventoriées varient entre 16 m² pour la plus petite et près de 2000 m² pour la plus grande. Au total, "
    "ce sont quatre plages de moins de 100 m², 15 de 100 à 500 m², cinq de 500 à 1100 m² et une de 1980 m² qui ont été "
    "investiguées. De plus, une attention particulière a été portée au substrat afin d’avoir une bonne répartition entre les plages "
    "de sable, de graviers et de galets. La séparation entre les différents substrats a été effectuée à l’œil en fonction de la "
    "granulométrie moyenne de la plage."
)

p14 = (
    "Chaque plage comprend deux sous-secteurs :"
)

l_two = (
    "la ligne d’eau (interface terre-eau + deux mètres de plage)",
    "la plage sèche"
)

p15 = (
    "A noter que la ligne d’eau se déplace au fil de l’année et des variations du niveau du lac."
)
content = add_paragraph_and_spacer(p13, content)


list_paragraph =  Paragraph(p14, body_text_style)



bulleted_list = make_a_list(l_two)

content.append(KeepTogether([list_paragraph, bulleted_list]))
content.append(Spacer(1, 12))
content = add_paragraph_and_spacer(p15, content)

a_caption = (
    "<b>Figure 4 :</b> Sous-secteurs des plages : 1) Ligne d’eau et 2) Plage sèche." 
    
)


image_path = "resources/images/pdfimages/beach_zones.jpg"  
image_width = 12 * cm
image_height = 9 * cm
image = Image(image_path, width=image_width, height=image_height)  

table_image = image_table(image, a_caption, image_width)
content.append(Spacer(1, 12))
content.append(table_image)
content.append(Spacer(1, 12))


content = add_section_title("Récolte, extraction et identification des microplastiques", content)
content = add_sub_section_title("Echantillonnage microplastique (300 µm à 5 mm)", content)

p16 = (
    "Les microplastiques, définis comme étant de taille inférieure à 5mm, ont été prélevés et analysés par "
    "les collaborateurs et collaboratrices de l’ASL ainsi que par les deux étudiantes de Master de l’UniGE. "
    "Un troisième travail de master en cours étudie les courants qui peuvent influencer le degré d’accumulation "
    "de microplastiques sur les plages."
)

content = add_paragraph_and_spacer(p16, content)
p17 = (
    "La stratégie d’échantillonnage est inspirée de celle de Gerdts (Gerdts, 2019). Sur chaque plage, "
    "une carotte de substrat est prélevée de manière aléatoire tous les 45 m² en moyenne à l’aide d’un profilé en métal "
    "de 10 cm de côté sur une profondeur de 5 cm (Figure 5). Cette profondeur correspond à celle utilisée dans le cadre d’études "
    "similaires (Faure et al., 2018), (Imhof et al., 2018), "
    "(Zbyszewski et al., 2014). Bien que concernant une unité de volume, il est usuel d’exprimer "
    "les résultats par surface. La méthode de prélèvement avec un profilé est adaptée de Bridson, (Bridson et al., 2020). "
    "La taille des carottes de 50 X 50 cm a été réduite à 10 X 10 cm afin de pouvoir augmenter le nombre "
    "d’échantillons analysés. La position géographique de chaque carotte est relevée au moyen d’un GPS. Les carottes "
    "sont transférées dans des boîtes de transport en verre et stockées à l’UNIGE en chambre froide à 4° C. "
)
content = add_paragraph_and_spacer(p17, content)
p18 = (
    "Les boîtes sont étiquetés de la manière suivante : 1. Pays, 2. Région, 3. numero de station, 4. numero d'échantillon. "
    "Les numéros d’échantillons sont définis de l’amont vers l’aval. "
    "Exemple: Baby Plage à Genève : CH GE 01.1; CH GE 01.2 ; CH GE 01.3 ..."

)

content = add_paragraph_and_spacer(p18, content)

# list_items = (
#     "Pays",
#     "Region",
#     "station",
#     "échantillon"
# )

# a_list = make_a_list(list_items)

# p19 = (
    
# )

# para19 = Paragraph(p19, body_text_style)

# agroup = KeepTogether([para18, Spacer(1,2), a_list, Spacer(1,2), para19])
# content.append(agroup)

p20 = (
    "Au total, 235 échantillons de 500 cm3 ont ainsi été prélevés en novembre 2021. "
    "Dans le cadre de l’étude, seuls 217 échantillons ont été traités et comptabilisés. "
    "18 sont conservés à l’Université de Genève en vue de futures études. "
)



content = add_paragraph_and_spacer(p20, content)

image_path = "resources/images/pdfimages/prelevement.jpg"  
image_width = 15 * cm
image_height = 11.2 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 5:</b> Prélèvement de 100 cm² de substrat (10 cm x 10 cm sur 5 cm de profondeur). "
    "Le profilé en métal est déposé en surface puis enfoncé à 5cm de profondeur. Le prélèvement est "
    "ensuite déposé dans une boîte de transport en verre et une feuille d’aluminium est placée entre l’échantillon et le "
    "couvercle pour éviter toutes contaminations."
)

table_image = image_table(image, a_caption, image_width)
content.append(Spacer(1, 12))
content.append(table_image)
content.append(Spacer(1, 12))

content = add_sub_section_title("Extraction et comptabilisation des microplastiques", content)

p21 = (
    "La manipulation des échantillons est effectuée dans de la verrerie rincée à l’eau ultrapure afin d’éviter toute nouvelle "
    "contamination par des plastiques (Bouzid et al., 2022). Dans un premier temps, les échantillons sont séchés "
    "dans un four à 60 °C. Les plastiques sont ensuite extraits par flottaisons dans une solution de Iodure de Sodium (NaI) à une "
    "densité de 1,7. Après 48 heures d’immersion, le surnageant est déposé sur des filtres en nitrate de cellulose d’une maille "
    "de 300 μm, puis placé dans une boîte en verre. Le matériel est constamment couvert d’une feuille d’aluminium pour éviter "
    "les contaminations. Les échantillons sont ensuite comptabilisés dans les locaux de l’ASL sous une cloche de plexiglas. "
)

content = add_paragraph_and_spacer(p21, content)


p22= (
    "L’identification des microplastiques se fait à l’aide d’une loupe binoculaire au grossissement 40x en respectant les trois "
    "règles définies par ([Hidalgo-Ruz et al., 2012](Hidalgo)). A savoir :"
)

para22 = Paragraph(p22, body_text_style)

list_items = (
    "il ne doit pas y avoir de cellules ou de structures organiques visibles",
    "pour les fibres, l’épaisseur doit être constante sur toute la longueur (sous réserve de présence de matière organique)"
    "la coloration doit être homogène."
)
new_list = make_a_list(list_items)
p22_list = KeepTogether([para22, new_list, Spacer(1,12)])




p23 = ("Selon ces recommandations, les particules comptabilisées ont été réparties en trois grands groupes :")
para23 = Paragraph(p23, body_text_style)

list_items = (
    "les fibres",
    "les particules souples (film, mousses)",
    "les particules dures (billes, fragments)"
)

new_list2 = make_a_list(list_items)

p23_list = KeepTogether([para23, new_list2, Spacer(1,12)])

content.append(p22_list)
content.append(p23_list)

image_path = "resources/images/pdfimages/microplastique.jpg"  
image_width = 14 * cm
image_height = 9.8 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 6 :</b> De gauche à droite: Fibre, particule souple et particule dure identifiées lors de la comptabilisation."
)

table_image = image_table(image, a_caption, image_width)
content.append(Spacer(1, 12))
content.append(table_image)
content.append(Spacer(1, 12))


content = add_sub_section_title("Limite de l’approche méthodologique", content)

p24 = (
    "Lors du traitement des échantillons à l’Université de Genève, de leur transport et de la comptabilisation "
    "à l’ASL des contaminations (au cours des manipulations en laboratoire, aériennes, liées à l’outillage, "
    "les vêtements...) peuvent intervenir. Des blancs de laboratoires ont donc été effectués (Tableau 1) en "
    "reproduisant toutes les étapes allant du traitement de l’échantillon avec le NaI à la comptabilisation aux locaux de l’ASL."
)

content = add_paragraph_and_spacer(p24, content)

# custom table
table_data = [
    ["Blancs", "Fibres transparentes", "Fibres noires", "Fibres bleues", "Fibres rouges"],
    ["Blanc 1", "15", "4", "0", "2"],
    ["Blanc 2", "39", "7", "4", "0"],
    ["Blanc 3", "4", "5", "5", "0"]
]
# Create the table
table = Table(table_data)
tab_cap = "<b>Tableau 1 :</b> Blancs de laboratoire effectués à l’UNIGE avec comptabilisation à l’ASL"
table_caption = Paragraph(tab_cap, caption_style)

# Style the table
table.setStyle(default_table_style)

table_with_caption = KeepTogether(Table([[table_caption], [table]]))

content.append(table_with_caption)

p25 = (
    "A noter que seules des fibres ont été retrouvées dans les blancs. Ces résultats ne sont "
    "pas surprenants au vu des risques de contamination aérienne. Avec une moyenne de 28 particules, "
    "comparée aux 189 particules comptabilisées en moyenne dans les échantillons, on peut considérer "
    "que la contamination est plutôt faible. Ces blancs ne sont pas soustraits des comptabilisations effectuées. "
)


content = add_paragraph_and_spacer(p25, content)

content = add_section_title("Récolte, extraction et identification des macroplastiques", content)

p26 = (
    "Un macroplastique est défini le plus souvent par la norme internationale comme étant un objet ou un fragment plastique "
    "plus grand que 5 mm.  Pour autant, dans le cadre de cette étude, le terme macroplastique sera utilisé pour l’ensemble "
    "des plastiques visibles à l’œil nu. Il comprend donc également les “grands” microplastiques entre 2 et 5 mm. La récolte "
    "et le recensement des macroplastiques ont été confiés à des personnes de la société civile, qui se sont spontanément "
    "inscrites pour se livrer à cette expérience de science participative. Une centaine de bénévoles très motivés ont ainsi "
    "appliqué un protocole scientifique précis sur les plages sélectionnées par l’ASL en respectant une surface et un temps "
    "de travail donné (30 secondes d’observation par m²) (Tableau 2). Cette précision est nécessaire pour limiter le biais "
    "induit par le grand nombre d’observateurs et d’observatrices."
)

content = add_paragraph_and_spacer(p26, content)

html_table = """
<table id="T_b0ff6">
  <thead>
    <tr>
      <th class="blank level0" > Plage </th>
      <th id="T_b0ff6_level0_col0" class="col_heading level0 col0" >Surface m²</th>
      <th id="T_b0ff6_level0_col1" class="col_heading level0 col1" >Bénévoles </th>
      <th id="T_b0ff6_level0_col2" class="col_heading level0 col2" > Ligne d'eau (min)</th>
      <th id="T_b0ff6_level0_col3" class="col_heading level0 col3" >Plage sèche (min)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_b0ff6_level0_row0" class="row_heading level0 row0" >Amphion</th>
      <td id="T_b0ff6_row0_col0" class="data row0 col0" >436</td>
      <td id="T_b0ff6_row0_col1" class="data row0 col1" >4</td>
      <td id="T_b0ff6_row0_col2" class="data row0 col2" >49</td>
      <td id="T_b0ff6_row0_col3" class="data row0 col3" >169</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row1" class="row_heading level0 row1" >Anthy-Séchex</th>
      <td id="T_b0ff6_row1_col0" class="data row1 col0" >245</td>
      <td id="T_b0ff6_row1_col1" class="data row1 col1" >2</td>
      <td id="T_b0ff6_row1_col2" class="data row1 col2" >28</td>
      <td id="T_b0ff6_row1_col3" class="data row1 col3" >95</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row2" class="row_heading level0 row2" >Aubonne</th>
      <td id="T_b0ff6_row2_col0" class="data row2 col0" >375</td>
      <td id="T_b0ff6_row2_col1" class="data row2 col1" >4</td>
      <td id="T_b0ff6_row2_col2" class="data row2 col2" >40</td>
      <td id="T_b0ff6_row2_col3" class="data row2 col3" >148</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row3" class="row_heading level0 row3" >Baby plage</th>
      <td id="T_b0ff6_row3_col0" class="data row3 col0" >513</td>
      <td id="T_b0ff6_row3_col1" class="data row3 col1" >5</td>
      <td id="T_b0ff6_row3_col2" class="data row3 col2" >58</td>
      <td id="T_b0ff6_row3_col3" class="data row3 col3" >199</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row4" class="row_heading level0 row4" >Bouveret</th>
      <td id="T_b0ff6_row4_col0" class="data row4 col0" >538</td>
      <td id="T_b0ff6_row4_col1" class="data row4 col1" >5</td>
      <td id="T_b0ff6_row4_col2" class="data row4 col2" >50</td>
      <td id="T_b0ff6_row4_col3" class="data row4 col3" >219</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row5" class="row_heading level0 row5" >Clarens</th>
      <td id="T_b0ff6_row5_col0" class="data row5 col0" >312</td>
      <td id="T_b0ff6_row5_col1" class="data row5 col1" >3</td>
      <td id="T_b0ff6_row5_col2" class="data row5 col2" >31</td>
      <td id="T_b0ff6_row5_col3" class="data row5 col3" >125</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row6" class="row_heading level0 row6" >Crans</th>
      <td id="T_b0ff6_row6_col0" class="data row6 col0" >11</td>
      <td id="T_b0ff6_row6_col1" class="data row6 col1" >1</td>
      <td id="T_b0ff6_row6_col2" class="data row6 col2" >6</td>
      <td id="T_b0ff6_row6_col3" class="data row6 col3" >-</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row7" class="row_heading level0 row7" >Cully</th>
      <td id="T_b0ff6_row7_col0" class="data row7 col0" >61</td>
      <td id="T_b0ff6_row7_col1" class="data row7 col1" >1</td>
      <td id="T_b0ff6_row7_col2" class="data row7 col2" >7</td>
      <td id="T_b0ff6_row7_col3" class="data row7 col3" >24</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row8" class="row_heading level0 row8" >Embouchure Versoix</th>
      <td id="T_b0ff6_row8_col0" class="data row8 col0" >166</td>
      <td id="T_b0ff6_row8_col1" class="data row8 col1" >2</td>
      <td id="T_b0ff6_row8_col2" class="data row8 col2" >18</td>
      <td id="T_b0ff6_row8_col3" class="data row8 col3" >65</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row9" class="row_heading level0 row9" >Excenevex</th>
      <td id="T_b0ff6_row9_col0" class="data row9 col0" >1977</td>
      <td id="T_b0ff6_row9_col1" class="data row9 col1" >17</td>
      <td id="T_b0ff6_row9_col2" class="data row9 col2" >87</td>
      <td id="T_b0ff6_row9_col3" class="data row9 col3" >901</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row10" class="row_heading level0 row10" >Gland</th>
      <td id="T_b0ff6_row10_col0" class="data row10 col0" >31</td>
      <td id="T_b0ff6_row10_col1" class="data row10 col1" >1</td>
      <td id="T_b0ff6_row10_col2" class="data row10 col2" >8</td>
      <td id="T_b0ff6_row10_col3" class="data row10 col3" >8</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row11" class="row_heading level0 row11" >Grangettes</th>
      <td id="T_b0ff6_row11_col0" class="data row11 col0" >321</td>
      <td id="T_b0ff6_row11_col1" class="data row11 col1" >3</td>
      <td id="T_b0ff6_row11_col2" class="data row11 col2" >37</td>
      <td id="T_b0ff6_row11_col3" class="data row11 col3" >123</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row12" class="row_heading level0 row12" >Hermance</th>
      <td id="T_b0ff6_row12_col0" class="data row12 col0" >383</td>
      <td id="T_b0ff6_row12_col1" class="data row12 col1" >4</td>
      <td id="T_b0ff6_row12_col2" class="data row12 col2" >54</td>
      <td id="T_b0ff6_row12_col3" class="data row12 col3" >137</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row13" class="row_heading level0 row13" >Lugrin</th>
      <td id="T_b0ff6_row13_col0" class="data row13 col0" >227</td>
      <td id="T_b0ff6_row13_col1" class="data row13 col1" >2</td>
      <td id="T_b0ff6_row13_col2" class="data row13 col2" >35</td>
      <td id="T_b0ff6_row13_col3" class="data row13 col3" >78</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row14" class="row_heading level0 row14" >Lutry</th>
      <td id="T_b0ff6_row14_col0" class="data row14 col0" >325</td>
      <td id="T_b0ff6_row14_col1" class="data row14 col1" >3</td>
      <td id="T_b0ff6_row14_col2" class="data row14 col2" >40</td>
      <td id="T_b0ff6_row14_col3" class="data row14 col3" >122</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row15" class="row_heading level0 row15" >Meillerie</th>
      <td id="T_b0ff6_row15_col0" class="data row15 col0" >14</td>
      <td id="T_b0ff6_row15_col1" class="data row15 col1" >1</td>
      <td id="T_b0ff6_row15_col2" class="data row15 col2" >7</td>
      <td id="T_b0ff6_row15_col3" class="data row15 col3" >-</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row16" class="row_heading level0 row16" >Pichette</th>
      <td id="T_b0ff6_row16_col0" class="data row16 col0" >235</td>
      <td id="T_b0ff6_row16_col1" class="data row16 col1" >2</td>
      <td id="T_b0ff6_row16_col2" class="data row16 col2" >21</td>
      <td id="T_b0ff6_row16_col3" class="data row16 col3" >97</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row17" class="row_heading level0 row17" >Port choiseul</th>
      <td id="T_b0ff6_row17_col0" class="data row17 col0" >437</td>
      <td id="T_b0ff6_row17_col1" class="data row17 col1" >4</td>
      <td id="T_b0ff6_row17_col2" class="data row17 col2" >72</td>
      <td id="T_b0ff6_row17_col3" class="data row17 col3" >146</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row18" class="row_heading level0 row18" >Préverenges</th>
      <td id="T_b0ff6_row18_col0" class="data row18 col0" >1123</td>
      <td id="T_b0ff6_row18_col1" class="data row18 col1" >10</td>
      <td id="T_b0ff6_row18_col2" class="data row18 col2" >76</td>
      <td id="T_b0ff6_row18_col3" class="data row18 col3" >486</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row19" class="row_heading level0 row19" >Rolle</th>
      <td id="T_b0ff6_row19_col0" class="data row19 col0" >954</td>
      <td id="T_b0ff6_row19_col1" class="data row19 col1" >8</td>
      <td id="T_b0ff6_row19_col2" class="data row19 col2" >62</td>
      <td id="T_b0ff6_row19_col3" class="data row19 col3" >415</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row20" class="row_heading level0 row20" >Saint-disdille</th>
      <td id="T_b0ff6_row20_col0" class="data row20 col0" >270</td>
      <td id="T_b0ff6_row20_col1" class="data row20 col1" >3</td>
      <td id="T_b0ff6_row20_col2" class="data row20 col2" >30</td>
      <td id="T_b0ff6_row20_col3" class="data row20 col3" >105</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row21" class="row_heading level0 row21" >Savonnière</th>
      <td id="T_b0ff6_row21_col0" class="data row21 col0" >401</td>
      <td id="T_b0ff6_row21_col1" class="data row21 col1" >4</td>
      <td id="T_b0ff6_row21_col2" class="data row21 col2" >40</td>
      <td id="T_b0ff6_row21_col3" class="data row21 col3" >160</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row22" class="row_heading level0 row22" >Tolochenaz</th>
      <td id="T_b0ff6_row22_col0" class="data row22 col0" >144</td>
      <td id="T_b0ff6_row22_col1" class="data row22 col1" >2</td>
      <td id="T_b0ff6_row22_col2" class="data row22 col2" >29</td>
      <td id="T_b0ff6_row22_col3" class="data row22 col3" >43</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row23" class="row_heading level0 row23" >Tougues</th>
      <td id="T_b0ff6_row23_col0" class="data row23 col0" >536</td>
      <td id="T_b0ff6_row23_col1" class="data row23 col1" >5</td>
      <td id="T_b0ff6_row23_col2" class="data row23 col2" >46</td>
      <td id="T_b0ff6_row23_col3" class="data row23 col3" >222</td>
    </tr>
    <tr>
      <th id="T_b0ff6_level0_row24" class="row_heading level0 row24" >Vidy</th>
      <td id="T_b0ff6_row24_col0" class="data row24 col0" >545</td>
      <td id="T_b0ff6_row24_col1" class="data row24 col1" >5</td>
      <td id="T_b0ff6_row24_col2" class="data row24 col2" >49</td>
      <td id="T_b0ff6_row24_col3" class="data row24 col3" >223</td>
    </tr>
  </tbody>
</table>
"""

soup = BeautifulSoup(html_table, 'html.parser')
table_data = []

for row in soup.find_all('tr'):
    table_data.append([cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])])


column_widths = [3.5* cm, 3 * cm, 3 * cm, 3 * cm, 3*cm]
reportlab_table = Table(table_data,colWidths=column_widths, repeatRows=1)

reportlab_table.setStyle(default_table_style)
reportlab_table.setStyle(TableStyle([
    # ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
]))
table_caption = "<b>Tableau 2 :</b> Nombre de bénévoles mobilisés par plage et temps d’échantillonnage par secteurs."
center_caption = ParagraphStyle(name='Caption', parent=styles['BodyText'], alignment=1, fontName='Helvetica-Oblique', fontSize=8, leading=10)
t_cap = Paragraph(table_caption, center_caption)

keep_together = [t_cap, Spacer(1, 4),reportlab_table]
content.append(KeepTogether(keep_together))
content.append(Spacer(1, 12))
# content.append(reportlab_table)
# content.append(Spacer(1, 12))
p27 =(
    "Le nombre d’intervenants, le secteur et le temps d’intervention doivent être rigoureusement respectés. En outre, la récolte "
    "des plus petits éléments (2 à 5 mm) demande une attention toute particulière."

)

content = add_paragraph_and_spacer(p27, content)

p28 = (
    "Le protocole de collecte et de saisie a été établi d’après les référentiels du Guide sur la surveillance des déchets marins dans "
    "les mers européennes publié par la Commission européenne en 2013 (Hanke et al., 2013). Son application permet de "
    "comparer les résultats obtenus sur les rives du Léman avec les autres études effectuées aux niveaux national et européen. "
)

content = add_paragraph_and_spacer(p28, content)

p29 = (
    "Le nombre de bénévoles envoyé sur chaque plage est proportionnel à sa taille. Des responsables ont été nommés pour chacune "
    "sur la base du volontariat et se sont portés garants de la bonne application du protocole et donc de la robustesse des résultats. "
    "Au nombre de 33, ils ont été formés à l’encadrement du travail de terrain ainsi qu’à la méthode de tri et dénombrement "
    "des récoltes qu’ils ont effectués à leur domicile. Les plages sélectionnées dans le cadre de l’étude étant toutes des plages publiques, "
    "les grands macroplastiques (bouteilles, jouets de sables…) sont régulièrement ramassés par les services communaux. "
)
content = add_paragraph_and_spacer(p29, content)

content = add_sub_section_title("Echantillonnage macroplastique (> 2 mm)", content)

p30 = (
    "La personne responsable de secteur accueille les bénévoles et leur transmet les informations sur le secteur à "
    "inventorier. Les macroplastiques récoltés dans les sous-secteurs (ligne d’eau et plage sèche) sont triés et comptabilisés séparément."
)

content = add_paragraph_and_spacer(p30, content)

p31 = (
    "La collecte des éléments de macroplastique a lieu sur la surface de plage prédéterminée au moyen d’un seau par "
    "bénévole ou de deux boîtes en plastique par personne responsable (Figure 7). Plus précisément il s’agit de "
    "récolter les objets et fragments déposés sur le sol, sans creuser, sur chacun des deux sous-secteurs :"
)

par31 = Paragraph(p31, body_text_style)

list_items = (
    "Ligne d’eau : à partir de la limite eau/sol, une largeur de plage adjacente de 2 m de part et d’autre",
    "Plage sèche : le reste de la plage"
)

alist = make_a_list(list_items)

list_and_para = KeepTogether([par31, alist])
content.append(list_and_para)

p32 = (
    "La collecte est exécutée par chaque groupe de bénévoles à une vitesse de 30 sec/m² sur l’ensemble du secteur "
    "(rythme lent permettant la récolte de très petits éléments). Cela signifie, par exemple, qu’une plage de 240 m², "
    "implique un travail de collecte de 2 heures, soit 30 minutes par personne pour un groupe de 4 bénévoles."
    "Chaque groupe de bénévoles se rend sur la plage qui lui est assignée une fois par saison durant une année. "
    "Les passages ont ainsi eu lieu en février-mars, avril-mai, juillet-août et octobre-novembre."
)

content = add_paragraph_and_spacer(p32, content)

# p33 = (
#     "Chaque groupe de bénévoles se rend sur la plage qui lui est assignée une fois par saison durant une année. "
#     "Les passages ont ainsi eu lieu en février-mars, avril-mai, juillet-août et octobre-novembre."
# )

# content = add_paragraph_and_spacer(p33, content)

image_path = "resources/images/pdfimages/benevoles.JPG"  
image_width = 13.9 * cm
image_height = 7.9 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 7 :</b> Récolte assidue des macroplastiques sur la plage de Préverenges. Photo ASL"
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
content.append(Spacer(1, 12))

content = add_sub_section_title("Tri et comptabilisation des macroplastiques", content)

p34 = (
    "Les plastiques récoltés sont réunis par sous-secteur et triés (Figure 8) selon la nomenclature reconnue au niveau européen "
    "(Hanke et al., 2013) présentée aux responsables lors des soirées de formation puis comptabilisés. Les données sont saisies "
    "dans l’application Net’Léman (netleman.app) de l’ASL qui permet de centraliser les données aux fins d’analyse.  Les données "
    "ont ensuite été vérifiées, puis analysées en vue de déterminer les proportions des différents types de plastique."
)

content = add_paragraph_and_spacer(p34, content)

image_path = "resources/images/pdfimages/macrotri.jpeg"  
image_width = 14 * cm
image_height = 9.3 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 7 :</b> Tri des macroplastiques récoltés sur la plage de Vidy selon des catégories d’objets (Annexe 1). Photo: Aleksandra Racz"
)

table_image = image_table(image, a_caption, image_width)
content.append(Spacer(1, 12))
content.append(table_image)
content.append(Spacer(1, 12))

keep_together = []

keep_together = add_sub_section_title("Limite de l’approche méthodologique", keep_together)

p35 = (
    "La récolte des macroplastiques et leur identification étant à la charge des bénévoles, un certain biais observateur "
    "peut interférer dans les mesures. La communication entre les bénévoles et les experts doit être très régulière."
)

keep_together=add_paragraph_and_spacer(p35, keep_together)
content.append(KeepTogether(keep_together))
chap5content = content.copy()
content.append(PageBreak())
# chap_5.build(content, onFirstPage=footer, onLaterPages=header_footer)

## Résultats

In [8]:
pdf_file = "chap_6.pdf"
chap_6 = new_document(pdf_file)

def extract_data_from_html(file_name):
    
    with open(file_name, 'r') as file:
        html_table = file.read()
    soup = BeautifulSoup(html_table, 'html.parser')
    table_data = []
    for row in soup.find_all('tr'):
        table_data.append([cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])])
    return table_data


# content = []

content = add_title("Résultats", content)

content = add_section_title("Microplastiques (0.3 à 5 mm)", content)

content = add_sub_section_title("Répartition des microplastiques sur les plages du Léman", content)

p36 = (
    "Sur la totalité des 217 échantillons analysés, les plages étudiées contiennent en moyenne 189 "
    "particules plastiques par 100 cm² avec un écart-type de 278. Parmi ces échantillons, une donnée "
    "sort du lot avec 3'143 particules comptabilisées à la plage de la Pichette. Ce n’est pas forcément "
    "pour autant une donnée aberrante. Elle signale la possibilité d’événements exceptionnels induits par "
    "la fréquentation du site ou par une accumulation par l’influence de facteurs environnementaux (vents, courants...)." 
)

p37 = (
    "L’élimination de cette donnée n’a que peu d’impact sur la médiane (passage de 126 à 125 pièces par échantillon). "
    "En revanche, la moyenne passe de 189 à 175 et l’écart type de 278 à 191 (Tableau 3). L’élimination de cette "
    "donnée permet donc une plus grande robustesse et stabilité des données. Cette donnée étant plus de deux fois "
    "supérieure à la deuxième donnée maximale, une contamination ponctuelle n’est pas exclue. Celle-ci ayant "
    "potentiellement pu avoir lieu avant le prélèvement."
)

content = add_paragraph_and_spacer(p36, content)
content = add_paragraph_and_spacer(p37, content)

table_one = extract_data_from_html('resources/tables/results_table_one.html')
table_two = extract_data_from_html('resources/tables/results_table_two.html')

caption_one = (
    "<b>Tableau 3 :</b> Valeurs globales du nombre de pièces identifiées comme étant du plastique par échantillon (particules/100 cm²). "
    "La distinction est faite entre la prise en compte de l'ensemble des données (<b>6.1a</b>) et uniquement celles sous le 99.9 ème percentile (<b>6.1b</b>)."
)

caption = Paragraph("Tableau 3a", caption_style)
t_one = Table(table_one, colWidths=[2*cm, 2*cm])
t_one.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_one]     
]

caption = Paragraph("Tableau 3b", caption_style)
t_two = Table(table_two, colWidths=[2*cm, 2*cm])
t_two.setStyle(default_table_style)
table_data_two = [
    [caption],     
    [t_two]     
]

# tablex = Table([[table_data_one, table_data_two]], colWidths=[7*cm, 7*cm])# colWidths=[9*cm, 4.5*cm, 4.5*cm])

tablexi = Table([[Paragraph(caption_one, caption_style), Table(table_data_one), Table(table_data_two)]], colWidths=[5*cm, 5*cm, 5*cm])
tablexi.setStyle(
    [
        ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'), 
        ('PADDING', (0, 0), (0, 0), 6),
        ('LEFTPADDING', (0, 0), (0, 0), 6),  
        ('RIGHTPADDING', (0, 0), (0, 0), 6), 
        ('TOPPADDING', (0, 0), (0, 0), 18), 
        ('BOTTOMPADDING', (0, 0), (0, 0), 6) 
    ]
)
content.append(tablexi)


image_path = "resources/images/pdfimages/carteMicro.jpg"  
image_width = 14 * cm
image_height = 9.3 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 8 :</b>  Répartition des teneurs en microplastiques autour du Léman. Densité moyenne de microplastiques par 100 cm² et par site d’échantillonnage."
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)

p37 = (
    "La Figure 8 permet de visualiser les plages où les microplastiques sont les plus abondants. Ce sont celles de la Pichette, sur la commune de Corseaux, de Port Choiseul (Versoix) et des Grangettes (Noville)."
)

content = add_paragraph_and_spacer(p37, content)

content = add_sub_section_title("Différence entre la ligne d’eau et la plage sèche", content)

p38 = (
    "Les médianes et moyennes sont nettement plus élevées sur la plage sèche que sur la ligne d’eau (Tableau 4, Figure 9). "
    "Cette différence peut s’expliquer par une accumulation possible sur la plage sèche, car elle est ponctuellement immergée lors "
    "de crues du lac, alors que la ligne d’eau est en permanence soumise aux vagues qui amènent et reprennent les éléments au fil du temps."
)

content = add_paragraph_and_spacer(p38, content)



image_path = "resources/images/pdfimages/results_figure_three.jpg"  
image_width = 8 * cm
image_height = 6 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 9 :</b> Les abondances en nombre de particules par 100 cm² sont généralement plus faibles sur la ligne d’eau que sur "
    "la plage sèche. Plusieurs concentrations extrêmes ont été recensées lors de l’étude, aussi bien sur la ligne d’eau que sur la "
    "plage sèche. Les trois concentrations les plus élevées observées (99ème percentile) étaient sur la plage sèche. "
)

table_image = image_table(image, a_caption, image_width)

table_three = extract_data_from_html('resources/tables/results_table_two.html')

a_caption = (
    "<b>Tableau 4 :</b> Teneurs en particules de microplastiques de l’ensemble des échantillons prélevés sur la ligne d'eau et la plage sèche. Résultats exprimés en particules/100 cm²."
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[2.5*cm, 2.5*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[5*cm])

table_figure = Table([[table_image, table_data_one]], colWidths=[10*cm, 6*cm])
table_figure.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'LEFT')])
content.append(table_figure)

p40 = (
    "Sur la ligne d’eau, les concentrations moyennes fluctuent entre 19 (Crans) et 884 particules par échantillon (Préverenges). "
    "Sur le secteur “plage sèche”, les échantillons contiennent entre 37 particules par 100 cm2 à Clarens et 1’492 à Port Choiseul. "
    "Ce dernier secteur se démarque très nettement des autres avec une médiane supérieure à 1'000 particules par échantillon  (Figure 10)."
)

content = add_paragraph_and_spacer(p40, content)

image_path = "resources/images/pdfimages/results_figure_one.jpg"  
image_width = 14 * cm
image_height = 7.7 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 10 :</b> Concentration en microplastiques sur les 25 plages étudiées et par position sur la plage (particules/100cm²). "
    "Une valeur extrême a été relevée avec 3'143 particules dans un échantillon à la plage de la Pichette. "
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)

p41 = (
    "Sur la ligne d’eau, les concentrations recensées sur la plage de l’Empereur aux Grangettes sortent du lot avec une médiane dépassant "
    "les 400 particules par 100 cm², contre 85 particules/100 cm² pour la totalité des sites. La plage des Grangettes est la seule où les "
    "particules recensées sont plus nombreuses sur la ligne d’eau que sur la plage sèche. Ceci pourrait s’expliquer par une forte exposition "
    "de la plage aux vents et courants. Le substrat de la plage de l’Empereur est également particulièrement fin sur la ligne d’eau, ce qui "
    "pourrait également expliquer une plus grande capacité de retenue des petites particules."
)

content = add_paragraph_and_spacer(p41, content)
content = add_sub_section_title("Identification des particules comptabilisées", content)

p42 = (
    "Parmi les 39’000 particules comptabilisées, la majorité sont des fibres (75%). Ensuite viennent les fragments souples (10%) "
    "et durs (9%) (Figure 10). Les mousses représentent 4% des particules et les films 2%. Seules quelques microbilles de plastiques "
    "ont été recensées. La prédominance des fibres est significative d’une pollution massive via les textiles. Cette répartition reste "
    "globalement stable sur l’ensemble des plages étudiées. La plage des Grangettes fait exception avec une moyenne de 42% de fragments "
    "souples (film et mousse) pour 56% de fibres. La grande présence de déchets, notamment de sagex, dans les zones de roselière à "
    "proximité de la plage pourrait expliquer cette surabondance d’éléments fragmentés. La répartition globale est proche de celle "
    "relevée par Constant (Constant et all., 2019) sur la côte du Golfe du Lion en Méditerranée. A l’inverse, "
    "l’étude de Faure et al.,(Faure et all., 2018), ne recensait que 10% de fibres pour 61% de fragments. Ceci peut "
    "s’expliquer en partie par la méthode d’identification : visuelle dans le cadre de Pla’stock et chimique dans le cadre de l’étude "
    "de (Faure et all., 2018)."
)

content = add_paragraph_and_spacer(p42, content)

image_path = "resources/images/pdfimages/microformes.jpg"  
image_width = 10 * cm
image_height = 7 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
   "<b>Figure 10 :</b> Répartition quantitative des particules comptabilisées par forme. Les fibres dominent largement en terme d’abondance."
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
content.append(Spacer(1,12))
content = add_sub_section_title("Analyse des différentes variables aléatoires", content)


p43 = (
    "Les concentrations médianes de particules de microplastiques des échantillons prélevés sur les plages de sable fin "
    "et de sable grossier sont plus élevées que celles qui proviennent de substrats d’une granulométrie plus grossière "
    "(gravier et cailloux) (Tableau 5). Pour autant, au vu de la distribution des données, la différence n’est pas "
    "significative. Cette observation ne confirme donc que partiellement l’hypothèse que les substrats plus fins retiennent "
    "plus de particules que les substrats plus grossiers."
)
content = add_paragraph_and_spacer(p43, content)
table_three = extract_data_from_html('resources/tables/summary-table-substrat.html')

a_caption = (
    "<b>Tableau 5 :</b> Teneurs en particules de microplastiques par échantillon en fonction du type de substrat"
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths = [2.5*cm, 1*cm, 2*cm, 2*cm, 2*cm, 1*cm, 4*cm, 2*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[17*cm])
content.append(table_data_one)

p44 = (
    "La fréquentation, la distance à un parking ou encore la situation de la plage ne semblent pas avoir d’effet majeur sur les teneurs en microplastiques "
)

content = add_paragraph_and_spacer(p44, content)

content = add_section_title("Macroplastiques (> 2 mm)", content)

content = add_sub_section_title("Répartition des macroplastiques sur les plages du Léman", content)

p45 = (
    "Au fil des quatre sessions d’échantillonnage sur les 25 plages sélectionnées, une moyenne de 0,77 "
    "pièce de plastique par m² a été retrouvée (Figure 11), la médiane est de 0,5 pièce/m². Une "
    "concentration maximale de 7.6 pièces par m² a été inventoriée sur la plage du Bouveret le 30 avril. "
    "Lors des 3 autres sessions sur la ligne d’eau, les concentrations étaient de 2,8, 4,5 et 0,03 pièces/m². "
)

content = add_paragraph_and_spacer(p45, content)

image_path = "resources/images/pdfimages/results_figure_five.jpg"  
image_width = 15 * cm
image_height = 15 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 11 :</b> Concentration en macroplastiques pour l’ensemble des recensements. des quatre sessions d’échantillonnage. "
    "A) Répartition des échantillons au fil des saisons. Les passages 3 et 4 ont été perturbés par les conditions météorologiques "
    "(pluie et vent) qui ont nécessité de décaler de quelques semaines certaines sessions afin de garantir la qualité des relevés. B) "
    "Boîte de Tukey : zone colorée = 25%-75% ; « moustache » = 1,5 x la distance interquartile. C) L’histogramme montre que les données "
    "ne sont pas réparties de manière normale. D) La fonction de répartition permet de déterminer la probabilité que la concentration "
    "recensée soit inférieur à une valeur donnée."
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
# content.append(Spacer(1,12))
# content = add_sub_section_title("Analyse des différentes variables aléatoires", content)
content.append(PageBreak())
p46 = (
    "La boîte de Tukey (Figure 11) permet de constater la forte concentration des mesures entre 0.18 et 0.97 pièces par m² "
    "avec ponctuellement des échantillons présentant des concentrations nettement plus élevées pouvant atteindre 7.6 pièces/m² "
    "lors d’une session de ramassage au Bouveret. Lors des sessions d’hiver et d’été, respectivement 4,5 et 2,8 pièces/m² ont "
    "été recensées. En s’intéressant également aux résultats des Grangettes avec des concentrations entre 2.25 et 5,6 pièces/m² "
    "sur la ligne d’eau et entre 1,8 et 3 pièces/m² sur la plage sèche, il est à relever que la majorité des valeurs extrêmes "
    "visibles sur la figure 6.8 concernent les plages du Bouveret et des Grangettes."
)

content = add_paragraph_and_spacer(p46, content)

image_path = "resources/images/pdfimages/cartemacro.png"  
image_width = 14 * cm
image_height = 9.8 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 12 :</b> Répartition des teneurs en macroplastiques autour du Léman. Concentration moyenne par m² et par site d’échantillonnage."
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)
content.append(Spacer(1,12))

p47 = (
    "Globalement, on constate que 12 sites sur les 25 sélectionnés ne contiennent moins de 0,5 pièces/m², 7 en accumulent entre 0,5 "
    "et 1 pièces/m², 4 entre 1-2 pièces/m² et 2 entre 2-3 pièces/m². En résumé, la moitié des sites étudiés sont clairement impactés "
    "par des accumulations de macro-déchets plastique et 2 plages sont très fortement impactées (Bouveret et Grangettes)."
)

content = add_paragraph_and_spacer(p47, content)

content = add_sub_section_title("Différence entre la ligne d’eau et la plage sèche", content)

p48 = (
    "Les résultats de la récolte de déchets sur les plages du Léman en 4 sessions durant l’année 2022 sont exprimés en nombre de pièces par m² prospecté."
)

content = add_paragraph_and_spacer(p48, content)

image_path = "resources/images/pdfimages/results_figure_seven.jpg"  
image_width = 8 * cm
image_height = 6 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 13 :</b> Les abondances en nombre de particules par 100 cm² sont généralement plus faibles sur la ligne d’eau que sur "
    "la plage sèche. Plusieurs concentrations extrêmes ont été recensées lors de l’étude, aussi bien sur la ligne d’eau que sur la "
    "plage sèche. Les trois concentrations les plus élevées observées (99ème percentile) étaient sur la plage sèche. "
)

table_image = image_table(image, a_caption, image_width)

table_three = extract_data_from_html('resources/tables/results_table_eight.html')

a_caption = (
    "<b>Tableau 6 :</b> Teneurs en déchets de macroplastiques de l’ensemble des échantillons prélevés sur la ligne d'eau et la plage sèche. Résultats exprimés en pièces/m²"

)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[2.5*cm, 2.5*cm, 2.5*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[7.5*cm])

table_figure = Table([[table_image, table_data_one]], colWidths=[8*cm, 7.5*cm])
table_figure.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'LEFT')])
content.append(table_figure)

p49 = (
    "Les moyennes et médianes des relevés sur la plage sèche et sur la ligne d’eau sont comparables entre les secteurs (Figure 13). "
    "Pour autant, les moyennes sont près de deux fois plus élevées que les médianes ce qui montre la grande variabilité des "
    "échantillonnages avec la présence régulière d’abondance élevée de macrodéchets (Tableau 6). Il est toutefois intéressant de "
    "constater le nombre de valeurs extrêmes sur la ligne d’eau. La valeur maximale est d’ailleurs plus de deux fois plus élevée sur "
    "la ligne d’eau que sur la plage sèche. Ceci pourrait s’expliquer par une plus grande influence des facteurs environnementaux "
    "(vagues, courants, vents). De plus, ces résultats contrastent avec ceux trouvés pour les microparticules de plastiques, dont la "
    "concentration était plus élevée sur le sous-secteur plage sèche (Figure 13). Les rôles joués par les facteurs environnementaux "
    "n’ont pas pu être testés dans le cadre de cette étude. En effet, pour les exploiter correctement, il faudrait avoir les données "
    "de courantologie locale ainsi que de l’angle et de la hauteur des vagues sur la plage. L’acquisition de ces données pourrait faire "
    "l’objet d’une étude spécifique à l’échelle d’un nombre restreint de plages."
)

content = add_paragraph_and_spacer(p49, content)

image_path = "resources/images/pdfimages/results_figure_six.jpg"  
image_width = 14 * cm
image_height = 7.7 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 14 :</b> Concentration moyenne des déchets récoltés sur chacune des 25 plages lémaniques étudiées en 2022. "
)

table_image = image_table(image, a_caption, image_width)
content.append(table_image)

p50 = (
    "Sur la ligne d’eau, la concentration moyenne varie de 0.12 pièce/m² à Tolochenaz à 3.74 pièces/m² au Bouveret et sur la plage sèche, "
    "de 0.12 pièces/m² à Anthy-sur-Léman et Aubonne à 2,52 pièces/ m² aux Grangettes (Figure 14). Au Bouveret, à Excenevex et à Lutry, "
    "une nette différence est visible entre la ligne d’eau et la plage sèche. Cette différence pourrait s’expliquer par des sessions de "
    "récoltes effectuées quelques jours après des vents du large ou après le passage des services de voirie."
)

content = add_paragraph_and_spacer(p50, content)

p51 = (
    "A Excenevex, où la configuration du site (plage très plate) présente une ligne d’eau qui peut fortement varier au fil des conditions "
    "de vent, l’abondance des déchets est plus élevée que sur la plage sèche. Dans ce cas notamment, l’intensité de l’entretien par la "
    "commune de ce site très fréquenté peut influencer les quantités de plastiques retrouvées. Les résultats font état des plastiques "
    "réellement présents sur les plages et non de la totalité s’y déposant. En effet, les services communaux effectuent un ramassage "
    "régulier des grands macroplastiques, ce qui explique le très faible nombre de bouteilles en PET (4) retrouvées sur l’ensemble des "
    "quatre passages. Il est donc possible que l’étude sous-estime la quantité de déchets arrivant sur les rives depuis le Léman. "
)

content = add_paragraph_and_spacer(p51, content)
content = add_sub_section_title("Types de macroplastiques sur les plages du Léman", content)

p52 = (
    "Pour la suite des analyses, les résultats seront exprimés en nombre de pièces par mètre linéaire de plage afin d’avoir des points "
    "de comparaison avec d’autres études traitant des macroplastiques."
)

content = add_paragraph_and_spacer(p52, content)
p53 = (
    "La valeur médiane du nombre de déchets par mètre (pcs/m) pour l’ensemble des 98 échantillons récoltés sur les 25 plages de "
    "l’étude à raison de 1 par saison est de 2 pièces/m. Cependant, les taux recensés peuvent varier en fonction de l’urbanisation "
    "du site et de son environnement proche, de sa fréquentation et du type de substrat."
)
content = add_paragraph_and_spacer(p53, content)

content = add_sub_section_title("Etude des principaux déchets récoltés", content)

p54 = (
    "La nature des objets et des fragments de plastique les plus souvent recensés et en plus grand nombre a été déterminée et conduit à "
    "la sélection des 11 premières catégories de déchets les plus abondants. Celles-ci englobent 89% des macro-plastiques récoltés, à "
    "savoir 24’156 déchets sur les 27’493 recueillis. Plus précisément, les critères de cette sélection étaient le nombre d’objets énumérés "
    "de chaque catégorie et/ou une fréquence minimale de 50% de présence dans les échantillonnages. Le Tableau 7 présente les différentes "
    "catégories de macro-plastiques retenues dans l’ordre d’abondance avec mention de leur importance relative par rapport au nombre total "
    "de déchets (colonne 2) et de leur fréquence d’échantillonnage (colonne 4)."
)

content = add_paragraph_and_spacer(p54, content)


table_three = extract_data_from_html('resources/tables/results_table_nine.html')

a_caption = (
    "<b>Tableau 7 :</b> Quantité et fréquence de recensement des 11 principales catégories de déchets "
    "récoltés et valeur médiane de leur abondance autour du Léman. En partant du postulat de base qu’il ne "
    "devrait pas y avoir de plastiques sur les plages, le taux de d’échec correspond à la fréquence (pourcentage "
    "d’échantillons) à laquelle l’objet est retrouvé. Codes: G75 à G80: fragments de plastiques non-identifiés, "
    "par classe de tailles (ex : G78 = 2,5 à 50 cm); G81 à G83: morceaux de polystyrène par classe de taille; "
    "G21 à G24: bouchon de bouteilles en PET, couvercle de pot en plastique ([Annexe IX](inventory))."
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[4.3*cm, 2*cm, 2*cm, 2*cm, 2.2*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[12.5*cm])
content.append(table_data_one)

p55 = (
    "Les fragments de plastiques de plus de 5 mm dont l’origine en tant qu’objet n’est pas identifiable "
    "constituent la large majorité des déchets récoltés (41%) et ils sont retrouvés dans 97% des échantillons. "
    "Viennent ensuite divers objets d’usage courant (i.e. mégots, emballages alimentaires, cotons-tiges) et "
    "des matériaux plus ou moins fragmentés utilisés dans la construction, l’isolation, la confection, etc."
)

content = add_paragraph_and_spacer(p55, content)

p56 = (
    "Les fibres plastiques du béton projeté qui sont actuellement regroupées avec les déchets de construction "
    "(G89) pourraient à terme être recensées avec leur propre code. Elles représentent 125 des 380 éléments "
    "comptabilisés parmi les déchets de construction."
)
content = add_paragraph_and_spacer(p56, content)

p57 = (
    "En s’intéressant de manière plus précise à la répartition des éléments identifiés sur la ligne d’eau et "
    "sur la plage sèche (Figure 14), il est intéressant de constater deux choses. Tout d’abord la forte présence "
    "des mégots sur la plage sèche et surtout la présence des cotons-tiges, déchets de construction et pellets parmi "
    "les déchets les plus retrouvés sur la plage sèche. Ceci montre qu’il y a un cheminement des déchets entre le lac "
    "et les rives. En effet, ces déchets arrivent dans le Léman par les eaux de ruissellement (déchets de constructions et pellets) "
    "et les eaux usées (cotons-tiges et pellets). Au fil de leur circulation dans le Léman, ils peuvent ensuite "
    "s’échouer sur les plages, sortir du Léman par le Rhône ou encore sédimenter. Les grands macroplastiques étant "
    "ramassés régulièrement sur les plages, seuls les plus petits déchets restent. La part d’éléments dont la source n’est "
    "pas identifiée s’élève à 52% sur la plage sèche et 55% sur la ligne d’eau. Ce pourcentage comprend l’ensemble des fragments "
    "plastiques (plastiques « durs » et polystyrène)."
)

content = add_paragraph_and_spacer(p57, content)

image_path = "resources/images/pdfimages/macroidentifie_LE.png"  
image_width = 10 * cm
image_height = 7 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 15 :</b> Distinction entre les éléments identifiés sur la ligne d’eau (gauche) et sur la plage sèche (droite). "
    "Cette figure ne prend pas en compte les fragments plastiques non identifiés visibles à l’œil nu qui représentent 52% des "
    "éléments retrouvés sur la plage sèche et 55% sur la ligne d’eau."
)

table_image = image_table(image, a_caption, image_width)

image_path = "resources/images/pdfimages/macroidentifie_PS.png"  
image_width = 10 * cm
image_height = 7 * cm
imagei = Image(image_path, width=image_width, height=image_height)  



new_table_data = Table([[table_image], [imagei]])
content.append(new_table_data)

p58 = (
    "De fortes fluctuations sont visibles entre les plages (Tableau 8a et Tableau 8b). Valeurs "
    "médianes des concentrations recensées sur les plages du Léman en nombre de pièces par mètre "
    "linéaire. Cette figure permet de faire ressortir les cas particuliers tels que les pellets industriels "
    "à Préverenges ou les mégots à Vidy. Une médiane de 0). Sans surprise, les fragments de plastiques de plus "
    "de 5 mm sont dominants sur la quasi-totalité des plages (code Gfrag). Parmi les éléments identifiables, les "
    "emballages de nourriture et les mégots sont très souvent retrouvés. Il est également intéressant de se pencher "
    "sur certains déchets particuliers. Par exemple, les cotons-tiges sont présents en grande quantité à Amphion, "
    "Clarens, au Bouveret et aux Grangettes, donc plutôt sur le haut et le grand lac. La proximité avec une STEP ne "
    "semble toutefois pas être déterminante. En effet, la plage des Grangettes est éloignée d’une STEP "
    "(mais proche de l’embouchure du Rhône) et les cotons-tiges en plastique sont interdits en France, ce qui indiquerait "
    "que ceux retrouvés à Amphion proviennent de plus loin."
)

content = add_paragraph_and_spacer(p58, content)

p59 = (
    "En prenant l’exemple des pellets industriels, il est intéressant de noter leur distribution particulière : on ne "
    "les retrouve que sur certaines plages. La plage de Préverenges, avec une médiane de 0,82 pellets par mètre de plage "
    "fait office d’exception autour du Léman. Les trois autres plages avec le plus de pellets sont le Bouveret (0,42 pièces/m), "
    "les Grangettes et Baby plage (chacune 0,27 pièces/m). "
)

content = add_paragraph_and_spacer(p59, content)

p60 = (
    "En s’intéressant aux différents sites, il est visible que les Grangettes et le Bouveret sont les sites où les "
    "déchets sont les plus abondants, et ce pour la plupart des objets. A l’inverse, la plage d’Excenevex, qui fait également "
    "partie des plages avec les plus grandes concentrations de plastiques, est principalement dominante au niveau des fragments "
    "de plastiques, dont l’origine n’est pas identifiable. Cette observation pourrait s’expliquer par la situation géographique "
    "d’Excenevex. La plage, située en fond de baie, fait certainement office de « cul de sac » pour les déchets. L’assiduité des "
    "services de voirie peut également être une piste. Il est possible que la plupart des déchets plus grands, et donc identifiables, "
    "aient été récoltés par la voirie. "
)

content = add_paragraph_and_spacer(p60, content)

table_three_x = extract_data_from_html('resources/charts/results_figure_eight_a.html')
a_caption = (
    "<b>Tableau 8a :</b> Quantité et fréquence de recensement des 11 principales catégories de déchets "
    "récoltés et valeur médiane de leur abondance autour du Léman. En partant du postulat de base qu’il ne "
    "devrait pas y avoir de plastiques sur les plages, le taux de d’échec correspond à la fréquence (pourcentage "
    "d’échantillons) à laquelle l’objet est retrouvé. Codes: G75 à G80: fragments de plastiques non-identifiés, "
    "par classe de tailles (ex : G78 = 2,5 à 50 cm); G81 à G83: morceaux de polystyrène par classe de taille; "
    "G21 à G24: bouchon de bouteilles en PET, couvercle de pot en plastique."
)

caption = Paragraph(a_caption, caption_style)
t_three = create_gradient_table(table_three_x)
t_three.setStyle(default_table_style_small_header)
table_data_one = [
    [caption],     
    [t_three]     
]

t_data_one = Table(table_data_one, colWidths = [25*cm])

rotated_table_with_caption = RotatedTableWithCaption(t_data_one,4, 160)

content.append(PageBreak())
content.append(rotated_table_with_caption)

table_four_x = extract_data_from_html('resources/charts/results_figure_eight_b.html')

a_caption = (
    "<b>Tableau 8b :</b> Quantité et fréquence de recensement des 11 principales catégories de déchets "
    "récoltés et valeur médiane de leur abondance autour du Léman. En partant du postulat de base qu’il ne "
    "devrait pas y avoir de plastiques sur les plages, le taux de d’échec correspond à la fréquence (pourcentage "
    "d’échantillons) à laquelle l’objet est retrouvé. Codes: G75 à G80: fragments de plastiques non-identifiés, "
    "par classe de tailles (ex : G78 = 2,5 à 50 cm); G81 à G83: morceaux de polystyrène par classe de taille; "
    "G21 à G24: bouchon de bouteilles en PET, couvercle de pot en plastique."
)

caption = Paragraph(a_caption, caption_style)
t_four = create_gradient_table(table_four_x)
t_four.setStyle(default_table_style_small_header)
table_data_three = [
    [caption],     
    [t_four]     
]

t_data_3 = Table(table_data_three, colWidths = [25*cm])

rotated_table_with_caption = RotatedTableWithCaption(t_data_3,4, 160)

content.append(PageBreak())
content.append(rotated_table_with_caption)
chap6content = content.copy()
content.append(PageBreak())
# chap_6.build(content, onFirstPage=footer, onLaterPages=header_footer)

## Discussion

In [9]:
pdf_file = "chap_7.pdf"
chap_7 = new_document(pdf_file)

# content = []

content = add_title("Discussion", content)

content = add_section_title("Microplastiques", content)

p61 = (
    "Sur l’ensemble des 25 plages étudiées, les résultats obtenus montrent une conentration moyenne en"
    "microplastiques de 17’500 ± 19’100 par m². Ce chiffre, issu des comptabilisations effectuées à la "
    "loupe binoculaire, peut être pondéré en se basant sur une étude réalisée en Méditerranée. En effet "
    "(Constant et al., 2019), ont déterminé des pourcentages d’identification des différentes particules. "
    "Les fibres y sont identifiées comme étant bien du plastique dans 37% des cas, les fragments 50%, "
    "les mousses 81% et les films 67%. En appliquant cette pondération aux données du projet Pla’stock, "
    "la contamination estimée en plastique serait de 7’600 ± 8’300 particules par m²."
)

content = add_paragraph_and_spacer(p61, content)

p62 = (
    "Le Tableau 10 ci-dessous, montre que les concentrations pondérées dans le cadre de l’étude Pla’stock "
    "(7’600 ± 8’300 particules/m²) sont plus de trois fois supérieures à la moyenne mesurée en 2014 sur le "
    "Léman et les autres lacs suisses. Ceci confirme les observations faites par Frei et al., (Frei et al., 2021), "
    "sur le lac de Lugano. Il aurait été intéressant d’avoir un état des lieux en 2014 de la pollution plastique sur "
    "les plages du lac de Lugano afin de comparer l’évolution entre les deux lacs."
)

content = add_paragraph_and_spacer(p62, content)

p63 =(
    "La comparaison avec une étude réalisée en 2019 en Méditerranée, sur quatre plages du Golfe du Lion, est encore "
    "plus flagrante (Constant et al., 2019). Contrairement aux autres résultats des autres études (Frei et al., 2021), "
    "(Faure et al., 2018), (Constant et al., 2019), les particules comptabilisées dans le cadre de la présente étude "
    "n’ont pas été caractérisées mais la valeur pondérée permet d’obtenir une concentration probable de particules de "
    "plastique et une meilleure comparaison avec d’autres études. La valeur non pondérée constitue la valeur maximale "
    "de particules de plastique dans les échantillons. "
    
)

conent = add_paragraph_and_spacer(p63, content)

p64 = (
    "Les méthodes de prélèvements sont similaires dans les différentes études. En revanche la taille des échantillons varie. "
    "Constant et al. et Faure et al. ont effectué des carottages de 0,5 x 0,5 m contre 0,1 x 0,1 m pour l’étude de Frei et al. "
    "et l’étude Pla’stock. Cette nouvelle taille semble devenir la norme pour les échantillonnages de microplastiques sur les plages."
    
)
content = add_paragraph_and_spacer(p64, content)
image_path = "resources/images/pdfimages/anterieure.png"  
image_width = 16.1 * cm
image_height = 7.3 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Tableau 9 :</b> Comparaison avec d’autres études ayant comptabilisé des microplastiques sur les plages (particules/m²)."
)

caption = Paragraph(a_caption, caption_style)

table_data_three = [
    [caption],     
    [image]     
]

t_data_3 = Table(table_data_three)

content.append(t_data_3)

p65 = (
    "Ce n’est pas une surprise de constater une augmentation des concentrations en microplastiques depuis 2014. Cette évolution "
    "peut s’expliquer par l’augmentation de la production mondiale de plastique, la fragmentation des déchets déjà présents dans "
    "le Léman ou encore par la consommation textile."
)

content = add_paragraph_and_spacer(p65, content)


content = add_section_title("Macroplastiques", content)

p66 = (
    "Sur les 25 plages étudiées par l’étude Pla’stock, la moyenne est de 3,42 ± 3,66 pièces par mètre linéaire. Ce résultat "
    "suggère que les quantités de certains déchets sauvages sont en baisse par rapport aux sessions d’échantillonnage précédentes "
    "sur les plages du Léman (Tableau 10). Le projet IQAASL (OFEV) est le plus similaire à l’étude Pla’stock. Les différences sont "
    "la supervision des bénévoles, le nombre de passages et le nombre de plages suivies. Dans le cadre de l’étude Pla’stock, les "
    "responsables de secteur ont reçu une formation théorique pour réaliser les inventaires alors que dans celui d’IQAASL, les bénévoles "
    "étaient accompagnés sur les plages. Les autres projets mentionnés ne sont pas des études à large échelle. Il s’agit plutôt de "
    "collectes spontanées par des groupes sur un nombre restreint de secteur. Pla’stock est la première étude à avoir analysé tant des "
    "plages suisses que françaises."
)

content = add_paragraph_and_spacer(p66, content)

p67 = (
    "Dans l’ensemble, cette étude fait état d’une valeur médiane de 2 pièces/mètre linéaire contre 3,7 à 4,4 dans le passé. "
    "Cette réduction est conforme aux conclusions de 2020 – 2021 combinant les résultats du suivi par SLR et de l’étude IQAASL. "
)

content = add_paragraph_and_spacer(p67, content)

table_three_x = extract_data_from_html('resources/tables/results_table_ten.html')

a_caption = (
    "<b>Tableau 10 :</b> Comparaison des résultats des campagnes de collecte de déchets sauvages au bord du Léman entre 2016 et 2022 "
    "(nombre de déchets par mètre de rivage, nombre d’échantillons et nombre total de déchets récoltés). La récolte de 2021 correspond "
    "à l’étude IQAASL (OFEV, 2021). Les campagnes de 2016 et de 2018 ont été menées respectivement par <a href='https://www.hammerdirt.ch/' color='blue'> hammerdirt </a> "
    "et par STOPPP le <a href='http://stopp.org/' color='blue'> Swiss Litter Report </a> "
)
caption = Paragraph(a_caption, caption_style)
t_three = create_gradient_table(table_three_x)
t_three.setStyle(default_table_style_small_header)
table_data_one = [
    [caption],     
    [t_three]     
]

t_data_one = Table(table_data_one, colWidths=[10*cm])

content.append(t_data_one)

p68 = (
    "Les déchets dont la concentration sur les plages a le plus fortement diminué sont ceux liés à la consommation personnelle (mégots de cigarettes, "
    "emballages de snacks et bouchons de bouteilles en plastique). La diminution marquée de la densité des cotons-tiges en plastique (0,03 pièce/mètre "
    "dans le cadre de l’étude Pla’stock contre 0,13 pièce/mètre lors de l’étude IQAASL pourrait être attribuée à l’interdiction de la vente de ces articles en France."
)

content = add_paragraph_and_spacer(p68, content)
content = add_section_title("Stock de plastiques sur les plages du Léman", content)

p69 = (
    "En vue de l’extrapolation des résultats de notre étude à l’ensemble des plages du Léman, la superficie des plages publiques a été estimée sur la base "
    "d’orthophotos et en se référant à la liste des plages de la CIPEL. Cette liste a été adaptée, certaines plages ayant été éliminées car ce sont "
    "uniquement des espaces de baignade dépourvus de grèves “naturelles” (enrochements, dalles en pierres ou en béton, etc.). Trois autres plages ont en "
    "revanche été incluses dans l’étude. Il s’agit d’une plage semi-privée à Crans (VD), de la plage de la Pichette sur la commune de Corseaux et de la "
    "plage de l’Empereur à Noville (Grangettes). Au total, la superficie des 90 plages retenues correspond à 68’400 m² pour un linéaire de 12’700 m. "
    "Selon cette estimation, la contamination plastique s’élève à 519’840’000 ± 567’720’000 particules sur les plages du Léman, basée sur la moyenne pondérée "
    "et 1’197’000’000 ± 1’306’400’000 sans pondération. "
)

content = add_paragraph_and_spacer(p69, content)

p70 = (
    "Pour les macroplastiques, sur les 12’700 m de plages publiques, en transposant la moyenne de 3,42 ± 3,66 pièces par mètre recensées dans "
    "le cadre du projet Pla’stock, il y aurait 43’400 ± 46’500 morceaux de plastiques sur les plages publiques du Léman. "
)

content = add_paragraph_and_spacer(p70, content)

content = add_section_title("Prédictions des abondances", content)

content = add_sub_section_title("Prédiction avec modèle de la grille Bayésienne", content)

p71 = (
    "La méthode de la grille Bayésienne estime les paramètres en divisant l’espace des paramètres en points, calculant les chances de chaque point "
    "sur une grille, et les combinant avec des estimations antérieures pour obtenir des estimations finales. Elle aide à estimer les paramètres "
    "et leur incertitude, mais peut être complexe à calculer. "
)

content = add_paragraph_and_spacer(p71, content)

content = add_sub_section_title("Microplastiques", content)
p72 = (
    "Les prédictions des abondances de microplastiques sur les plages du Léman permettent de visualiser ce qui est censé être réellement "
    "présent sur les plages en fonction des échantillonnages réalisés en 2021 dans le cadre de cette étude. Dans un premier temps, "
    "la prédiction d’abondance a été effectuée pour l’ensemble des plages, toutes conditions confondues (Figure 16)."
)

content = add_paragraph_and_spacer(p72, content)


image_path = "resources/images/pdfimages/micro_predictions_tous_histo.jpeg"  
image_width = 8.9 * cm
image_height = 5.6 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 16 :</b> Prédiction d'abondance par approximation Bayésienne"
)

table_image = image_table(image, a_caption, image_width)
table_image.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'RIGHT')])

table_three = extract_data_from_html('resources/tables/micro_prediction_tous_conditions.html')

a_caption = (
    "<b>Tableau 11 :</b> Prédiction d'abondance par approximation Bayésienne"
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[1.5*cm, 2.5*cm, 2.5*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[6.5*cm])
table_figure = Table([[table_image, table_data_one]], colWidths=[9*cm, 8*cm])
table_figure.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'LEFT')])
content.append(table_figure)

p73 = (
    "Les substrats ont été regroupés en deux catégories. Les sables comprennent les sables fins et grossiers. Les graviers "
    "comprennent les graviers et les cailloux. Les futurs relevés permettront d’alimenter le modèle de prédiction et d’en "
    "augmenter la robustesse. A ce stade, la prédiction indique qu’en allant se promener sur une plage de sable, nous trouverons "
    "en moyenne 200 particules de plastiques (valeur non pondérée), avec 99% des échantillons contenant moins de 1031 particules "
    "par échantillons de 100 cm². En s’intéressant à la distinction entre les types de substrats, il est prévisible de trouver "
    "en moyenne plus de particules sur les plages dont le substrat est grossier (graviers et cailloux) que sur celles qui sont "
    "de sable. La dispersion des mesures est très grande sur les plages de graviers/galets avec des abondances pouvant varier de "
    "1 à 1446 particules/100 cm² dans 99% des cas. A noter que les prédictions prévoient une différence entre les substrats sableux "
    "et les substrats grossiers moindre que celle mesurée in situ (Tableau 11)."
)

content = add_paragraph_and_spacer(p73, content)

image_path = "resources/images/pdfimages/micro_predictions_sable_histo.png"  
image_width = 8.9 * cm
image_height = 5.6 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 17 :</b> Prédiction d'abondance en nombre de particules/100 cm² sur les substrats sableux."
)

table_image = image_table(image, a_caption, image_width)
table_image.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'RIGHT')])

table_three = extract_data_from_html('resources/tables/micro_sables_prediction.html')

a_caption = (
    "<b>Tableau 12 :</b> Prédiction d'abondance en nombre de particules/100 cm² sur les substrats sableux."
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[1.5*cm, 2.5*cm, 2.5*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[6.5*cm])
table_figure = Table([[table_data_one, table_image]], colWidths=[9*cm, 8*cm])
table_figure.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'LEFT')])
content.append(table_figure)
content.append(Spacer(1, 12))




image_path = "resources/images/pdfimages/micro_predictions_gravier_histo.jpeg"  
image_width = 8.9 * cm
image_height = 5.6 * cm
image = Image(image_path, width=image_width, height=image_height)  

a_caption =(
    "<b>Figure 18 :</b> Prédiction d'abondance en nombre de particules/100 cm² sur les plages de graviers."
)

table_image = image_table(image, a_caption, image_width)
table_image.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'RIGHT')])

table_three = extract_data_from_html('resources/tables/micro_graviers_prediction.html')

a_caption = (
    "<b>Tableau 13 :</b> Prédiction d'abondance en nombre de particules/100 cm² sur les plages de graviers."
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[1.5*cm, 2.5*cm, 2.5*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[6.5*cm])
table_figure = Table([[ table_data_one, table_image]], colWidths=[9*cm, 8*cm])
table_figure.setStyle([('VALIGN', (0, 0), (-1, -1), 'TOP'), ('ALIGN', (0, 0), (-1, -1), 'LEFT')])
content.append(table_figure)

p74 = (
    "Sur la base des différentes sessions d’échantillonnage de ces dernières années, il est possible de prédire "
    "les abondances de microplastiques qui seront retrouvées sur les plages du Léman en 2024. Les substrats ont été "
    "regroupés en deux catégories. Les sables comprennent les sables fins et grossiers et les graviers comprennent "
    "les graviers et les cailloux. Les futurs relevés permettront d’alimenter le modèle de prédiction. A ce stade, "
    "la prédiction indique qu’en allant se promener sur une plage de sable, nous trouverons en moyenne 188 particules "
    "de plastiques, avec la possibilité d’avoir des contaminations extrêmes jusqu’à 879 particules par échantillons de 100 cm². "
)

content.append(Spacer(1, 12))

content = add_paragraph_and_spacer(p74, content)

content = add_sub_section_title("Macroplastiques", content)

p75 = (
    "Sur la base des différentes sessions d’échantillonnage de ces dernières années, il est possible de prédire les "
    "abondances de macroplastiques qui seront retrouvées sur les plages du Léman en 2024. Le but est de pouvoir combiner, "
    "au sein de ce modèle, les données récoltées dans le cadre de Pla’stock et celles issues d’études plus anciennes. "
    "De cette manière, il sera possible de connaître la quantité et le type de plastiques qui seront trouvés en allant "
    "se promener sur une plage donnée. La prédiction se base uniquement sur les collectes effectuées et ne prend pas en "
    "compte les ramassages effectués par les voiries. "
)

content = add_paragraph_and_spacer(p75, content)

table_three = extract_data_from_html('resources/tables/conclusion_predictions_table_macro_all.html')

a_caption = (
    "<b>Tableau 14 :</b>  Prédiction d'abondance en nombre de pièces de macroplastiques par mètre linéaire."
)

caption = Paragraph(a_caption, caption_style)
t_three = Table(table_three, colWidths=[1.5*cm, 2*cm, 2*cm, 2*cm, 2*cm, 2*cm, 2*cm, 2*cm])
t_three.setStyle(default_table_style)
table_data_one = [
    [caption],     
    [t_three]     
]

table_data_one = Table(table_data_one, colWidths=[15.5*cm])

content.append(table_data_one)
chap7content = content.copy()

# chap_7.build(content, onFirstPage=footer, onLaterPages=header_footer)
content.append(PageBreak())

## Conclusion

In [10]:
pdf_file = "plastock_review.pdf"
chap_8 = new_document(pdf_file)

# content = []



content = add_title("Conclusion", content)

p76 = (
    "En conclusion de cette étude, une grande disparité de concentration est observable entre les différentes plages, tant pour les micro "
    "que pour les macroplastiques. Une étude similaire à l’échelle de chaque plage révélerait probablement la même hétérogénéité, compte tenu "
    "des zones d’accumulation observées sur différentes plages. Parmi les variables relevées (fréquentation, substrat, situation et distance à un parking), "
    "aucune ne se démarque comme étant déterminante pour expliquer la pollution plastique des plages du Léman et ce, bien que des variations aient été "
    "constatées. Cependant, une plage de sable, à proximité d’un parking et très fréquentée, risque d’être plus fortement impactée qu’une plage "
    "isolée avec galets. Entre la ligne d’eau et la plage sèche, il semblerait que l’accumulation de plastique ait plutôt lieu sur la plage sèche. "
    "Ceci pourrait s’expliquer par un dépôt des plastiques lors de forts vents du large qui sont ensuite stockés dans le substrat. A l’inverse, la "
    "ligne d’eau est continuellement exposée à la dynamique lacustre et l’importance de ces dépôts peut fortement varier en fonction des vents récents. "
    "Ces éléments devront être testés lors de futures études afin de pouvoir retracer au mieux le cheminement des plastiques dans le Léman et ainsi, dans "
    "la mesure du possible, en retracer la source. Le fait que, pour les macroplastiques, il n’y ait pas de variations visibles entre la ligne d’eau et "
    "la plage sèche peut s’expliquer par la fréquence des nettoyages effectués de manière très variable par les services communaux. Le transfert de "
    "déchets entre le lac et les rives est tout de même rendu visible par les types de déchets retrouvés sur la plage sèche (cotons-tiges, pellets, déchets de construction). "
)

content = add_paragraph_and_spacer(p76, content)


p77 = (
    "Les comparaisons avec des études antérieures montrent une nette augmentation des concentrations en microplastiques depuis l’étude de Faure et al. "
    "avec une prédominance des fibres. En revanche, les concentrations en macroplastiques tendent à diminuer légèrement. Ces évolutions seront intéressantes "
    "à suivre lors de futures études. Concernant les microplastiques identifiés, les fibres anthropiques sont largement dominantes, ce qui démontre le rôle "
    "important de l’abrasion des textiles dans l’introduction de microplastiques dans l’environnement. "
)

content = add_paragraph_and_spacer(p77, content)

p76 = (
    "En ce qui concerne les macroplastiques, il convient de noter l’importance de la fragmentation observée sur les déchets récupérés. En effet, "
    "plus de la moitié des objets triés n’étaient plus identifiables, preuve que les macroplastiques constituent la source de microplastiques "
    "secondaires. Parmi les éléments identifiables les plus fréquemment rencontrés, on retrouve les mégots, les emballages alimentaires et les pellets industriels. "
    
)

content = add_paragraph_and_spacer(p76, content)

p77 = (
    "Des tests sur les apports des principaux cours d’eau sont en cours de réalisation dans le cadre d’un travail de master avec l’UNIGE. "
    "Il serait également intéressant de pouvoir étudier l’influence des vents et des courants sur la distribution des plastiques autour du lac. "
    "L’exemple des Grangettes et du Bouveret qui subit l’effet combiné de la proximité du Rhône et des contre-courants ramenant les déchets vers "
    "les rives, illustre bien l’influence que peuvent avoir ces variables. Il serait également important de pouvoir identifier les différents "
    "vecteurs et sources de pollution pour les réduire au maximum. "
)

content = add_paragraph_and_spacer(p77, content)


p78 = (
    "Par ailleurs, grâce à l’identification des différents déchets recensés sur les plages du Léman, des mesures politiques pourraient être prises "
    "pour réduire, voire supprimer, le rejet de certains d’entre eux dont le cheminement dans l’environnement est connu. "
)

content = add_paragraph_and_spacer(p78, content)
chap8content = content.copy()

content.append(PageBreak())

In [11]:
r1 = (
    "<b>Guidance on Monitoring of Marine Litter in European Seas</b>, Galgani et all,  EUR 26113. Luxembourg (Luxembourg): "
    "Publications Office of the European Union; 2013. JRC83985 <a href='https://publications.jrc.ec.europa.eu/repository/handle/JRC83985' color='blue'>JRC83985</a>"
)

r2 = (
    "<b>Guidance on the Monitoring of Marine Litter in European Seas An update to improve the harmonised monitoring of "
    "marine litter under the Marine Strategy Framework Directive</b>,  Galgani et all., EUR 31539 EN, Publications Office of the European Union, Luxembourg, 2023, "
    "ISBN 978-92- 68-04093-5, doi:10.2760/59137, JRC133594 <a href='https://literatur.thuenen.de/digbib_extern/dn067364.pdf' color='blue'> JRC133594 </a>"
)

r3 = (
    "<b>A European threshold value and assessment method for macro litter on coastlines</b>, Publications Office of the "
    "European Union, Van Loon, et all , ISBN 978-92-76-21444-1, doi:10.2760/54369, JRC121707 <a href='https://publications.jrc.ec.europa.eu/repository/handle/JRC121707' color='blue'> JRC121707 </a>"
)

content = add_title("Références", content)
content = add_paragraph_and_spacer(r1, content)
content = add_paragraph_and_spacer(r2, content)
content = add_paragraph_and_spacer(r3, content)

r4 = (
    "<b>Improving big citizen science data: Moving beyond haphazard sampling</b>, Amano, Tatsuya and Lamming, Jonathan D. L. and Sutherland, William J, "
    "PLoS Biology, 2016, doi: 10.1371/journal.pbio.1002413, <a href='https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.1002413' color='blue'>1002413</a>."
)

r5 = (
    "<b>Marine Litter Assessment in the Adriatic and Ionian Seas</b>, Vlachogianni et all, 2017. IPA-Adriatic DeFishGear Project, MIO-ECSDE, HCMR and ISPRA. pp. 168 (ISBN: 978-960-6793-25-7)"
)

r6 = (
    "<b>Assessing marine litter on Mediterranean beaches: Filling in the knowledge gaps via a participatory-science initiative</b>, Vlachogianni, Th., "
    "2019, MIO-ECSDE, <a href='https://mio-ecsde.org/project/assessing-marine-litter-on-mediterranean-beaches-filling-in-the-knowledge-gaps-via-a-participatory-science-initiative-vlachogianni-th-mio-ecsde-2019/' color='blue'>MIO-ECSDE</a>"
)

for reference in [r4, r5, r6]:
    content = add_paragraph_and_spacer(reference, content)

r7 = (
    "<b>(Micro) plastic fluxes and stocks in Lake Geneva</b>, Boucher, J., Faure, F., Pompini, O., Plummer, Z., Wieser, O., & de Alencastro, L. F. (2019).  "
    "basin. TrAC Trends in Analytical Chemistry, 112, 66-74."
)

r8 = (
    "<b>Evaluation de la pollution par les plastiques dans les eaux de surface en Suisse</b>, Faure, F., De Alencastro, L. F. (2018)."
)

r9 = (
    "<b>Defining the Baselines and standards for Microplastics Analyses in European waters: Final report Project BASEMAN</b>, "
    "Gerdts, G. (2019). <a href='https://repository.oceanbestpractices.org/handle/11329/1205' color='blue'>Ocean best practices</a>"
)

for reference in [r7, r8, r9]:
    content = add_paragraph_and_spacer(reference, content)

r10 = (
    "<b>Beached microplastics in the Northwestern Mediterranean Sea</b>, Mel Constant, Philippe Kerhervé, Morgan Mino-Vercellio-Verollet, "
    "Marc Dumontier, Anna Sànchez Vidal, Miquel Canals, Serge Heussner, Marine Pollution Bulletin, Volume 142, 2019, Pages 263-273, "
    "ISSN 0025-326X, <a href='https://doi.org/10.1016/j.marpolbul.2019.03.032' color='blue'> Beached microplastics </a>"

)

r11 = (
    "<b>Assessment of microplastic contamination on lakeshores of Lake Lugano</b>, Frei, G., Peduzzi, S., Stoll, S., & Bruder, A."
)

r12 = (
    "<b>Comparison of the distribution and degradation of plastic debris along shorelines of the Great Lakes, North America</b>, Zbyszewski, M., Corcoran, P. "
    "L., & Hockin, A. (2014).  Journal of Great Lakes Research, 40(2), 288-299."
)

r13 = (
    "<b>Variation in plastic abundance at different lake beach zones-A case study</b>, Imhof, H. K., Wiesheu, A. C., Anger, P. M., Niessner, R., Ivleva, N. P., & Laforsch, "
    "C. (2018). Variation in plastic abundance at different lake beach zones-A case study. Science of the Total Environment, 613, 530-537. "
)

r14 = (
    "<b>Microplastic contamination in Auckland (New Zealand) beach sediments</b>, Bridson, J. H., Patel, M., Lewis, A., Gaw, S., & Parker, K. (2020).  Marine Pollution Bulletin, 151, 110867."
)

for reference in [r10, r11, r12, r13, r14]:
    content = add_paragraph_and_spacer(reference, content)

content.append(PageBreak())

r15 = (
    "<b>Quantification of Microplastics by Pyrolysis Coupled with Gas Chromatography and Mass Spectrometry in Sediments: Challenges and Implications</b>.Bouzid N, Anquetil C, Dris R, Gasperi J, "
    "Tassin B, Derenne S. Microplastics. 2022; 1(2):229-239. <a href='https://doi.org/10.3390/microplastics1020016' color='blue'> Microplastics </a>"
)

r16 = (
    "<b>Identification, quantification and analysis of observable anthropogenic litter along Swiss lake systems</b>, OFEV. 2021. "
    "<a href='https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/' color='blue'> IQAASL </a> "
)

r17 = (
    "<b>Swiss Litter Report</b>, Gabrielle, Kuhl; Pascal, Blahrer; 2018; <a href='http://stoppp.org/' color='blue'> Stopp research </a>"
)

for reference in [r15, r16, r17]:
    content = add_paragraph_and_spacer(reference, content)


r18 = (
    "<b>Microplastic sources, formation, toxicity and remediation: a review.</b> Osman, A. I., Hosny, M., Eltaweil, A. S., Omar, S., Elgarahy, A. M., Farghali, "
    "M., ... & Akinyede, K. A. (2023). Environmental Chemistry Letters, 21(4), 2129-2169."
)

r19 = (
    "<b>Distribution and abundance of small plastic debris on beaches in the SE Pacific (Chile): a study supported by a citizen science project.</b> Hidalgo-Ruz, V., & Thiel, "
    "M. (2013). Marine environmental research, 87, 12-18."
)

r20 = (
    "<b>A large-scale investigation of microplastic contamination: abundance and characteristics of microplastics in European beach sediment.</b> Lots, F. A., Behrens, P., Vijver, M. G., "
    "Horton, A. A., & Bosker, T. (2017). Marine pollution bulletin, 123(1-2), 219-226."
)

for reference in [r18, r19, r20]:
    content = add_paragraph_and_spacer(reference, content)

content = add_section_title("Mathématiques et informatique", content)

r21 = (
    "<b>scipy: Fundamental Algorithms for Scientific Computing in Python</b>, _Virtanen et al_, Nature Methods, 2020, doi: 10.1038/s41592-019-0686-2."
)

r22 = (
    "<b>pandas</b>, The pandas development team, Python software, 2020, doi: 10.5281/zenodo.3509134, "
    "<a href='https://doi.org/10.5281/zenodo.3509134' color='blue'>pandas</a>"
)

r23 = (
    "<b>Python 3 Reference Manual</b>, Guido Van Rossum and Fred L. Drake, CreateSpace, 2009, isbn: 1441412697."
)

r24 = (
    "<b>Bayesian Data Analysis - Third edition</b>, Andrew Gelman, Columbia university, 2021, doi: 10.1201/b16018. "
    "<a href='http://www.stat.columbia.edu/~gelman/book/BDA3.pdf' color='blue'> Bayesian Data Analysis </a>" 
)
r25 = (
    "<b>Think Bayes 2ed</b>, Alan Downey, Green Tea Press, 2021, <a href='http://allendowney.github.io/ThinkBayes2/' color='blue'> Think Bayes </a>"
)

r26 = (
    "<b>Regression analysis of count data. 2nd ed</b>, _A. Cameron and Pravin Trivedi, Technometrics, 1999, doi: 10.1017/CBO9780511814365."
)

r27 = (
    "<b>Jupyter Book</b>, Jupyter Book, 2021, <a href='https://zenodo.org/record/4539666' color='blue'> Jupyter Book</a>"
)

r28 = (
    "<b>Bayesian Methods for Hackers</b>, Cameron Davidson-Pilon, Addison-Wesley Professional, 2021, doi: 10.5555/2851115"
    "<a href='https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers' color='blue'> Bayesian-Methods-for-Hackers</a>"
)

for reference in [r21, r22, r23, r24, r25, r26, r27, r28]:
    content = add_paragraph_and_spacer(reference, content)

r29 = (
    "<b>Bayes Rules! An Introduction to Applied Bayesian Modeling</b>, Alicia Johnson, Chapman and Hall, 2021, "
    "isbn: 9780367255398. <a href='https://www.bayesrulesbook.com/chapter-1.html' color='blue'>Bayes Rules </a>"
)

r30 = (
    "<b>scikit-learn: Machine Learning in python</b>, Pedregosa et al, Journal of Machine Learning Research, vol 12 2011"
)

r31 = (
    "<b>Matplotlib: A 2D graphics environment</b>, Hunter, J. D., IEEE COMPUTER SOC, 2007, doi: 10.1109/MCSE.2007.55."
)

r32 = (
    "<b>Bayesian Analysis with Python - Second Edition</b>, Martin Oswaldo, PackT, 2020, isbn: 9781789341652."
)

r33 = (
    "<b>Probability Theory: The logic of science</b>, Edwin Jaynes, Washington University, 2003, isbn: 0521592712."
)

r34 = (
    "<b>Resampling</b>, _Wikepedia_, 2021, <a href='https://en.wikipedia.org/wiki/Resampling_(statistics)' color='blue'> Resampling Wikepedia </a>"
)

for reference in [r29, r30, r31, r32, r33, r34]:
    content = add_paragraph_and_spacer(reference, content)

chap_8.build(content, onFirstPage=footer, onLaterPages=footer)  

In [12]:
from PyPDF2 import PdfMerger

# List of PDF files to be combined
pdf_files = [
    "intro.pdf",
    "plastock_review.pdf",
  
]

# Create a PdfMerger object
merger = PdfMerger()

# Iterate over the list of PDF files and append each to the merger
for pdf in pdf_files:
    merger.append(pdf)

# Write out the combined PDF
with open("combined_output.pdf", "wb") as output_pdf:
    merger.write(output_pdf)

# Close the merger object
merger.close()

